In [ ]:
import sys
import cmath
import math
import os
import h5py
import matplotlib.pyplot as plt   # plots
from matplotlib.ticker import MaxNLocator
import numpy as np
import time
import warnings
import glob
from liblibra_core import *
import util.libutil as comn
from libra_py import units
import models
import libra_py.dynamics.tsh.compute as tsh_dynamics
import libra_py.dynamics.tsh.plot as tsh_dynamics_plot
import libra_py.data_savers as data_savers

from recipes import fssh, fssh2, fssh3, gfsh

import libra_py.models.GLVC as GLVC


warnings.filterwarnings('ignore')

colors = {}
colors.update({"11": "#8b1a0e"})  # red       
colors.update({"12": "#FF4500"})  # orangered 
colors.update({"13": "#B22222"})  # firebrick 
colors.update({"14": "#DC143C"})  # crimson   
colors.update({"21": "#5e9c36"})  # green
colors.update({"22": "#006400"})  # darkgreen  
colors.update({"23": "#228B22"})  # forestgreen
colors.update({"24": "#808000"})  # olive      
colors.update({"31": "#8A2BE2"})  # blueviolet
colors.update({"32": "#00008B"})  # darkblue  
colors.update({"41": "#2F4F4F"})  # darkslategray

clrs_index = ["11", "21", "31", "41", "12", "22", "32", "13","23", "14", "24"]


# Check the "decoherence time" for MFSD, SDM, and DISH. 

## Schwartz 1

In [ ]:
xls = [50.0]
ntraj = 1000
istate = 1
elec_int = 2
rep = 1
dt = 5.0
dpi = 300
print(f'{dt} {elec_int}')
eps_vals = [0.01, 0.05, 0.1, 0.4, 1.0,10.0, 50.0, 100.0, 500.0, 1000.0]
A_vals = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0, 
          1000000.0, 10000000.0, 100000000.0, 1000000000.0]

In [ ]:
F = h5py.File(f'all_methods_dec_wigner_saikat/fssh__ntraj_{ntraj}_iter_0_dt_{dt}_istate_{istate}_rep_{rep}_elec_int_{elec_int}/mem_data.hdf')
time_vec = np.array(F['time/data'][:])*units.au2fs/1000
ml_mctdh = np.loadtxt('reference/ML-MCTDH_n32.dat')
# simulation_time_vec = np.arange(x.shape[0]) * units.au2fs * dt
simulation_time_vec = time_vec * 1000
reference_time_vec = ml_mctdh[:, 0]

# Use searchsorted for efficient nearest-neighbor indexing
indices = np.searchsorted(simulation_time_vec, reference_time_vec)

# Adjust indices to find the *closest* not just the next one
indices = np.clip(indices, 1, len(reference_time_vec) - 1)
left = simulation_time_vec[indices - 1]
right = simulation_time_vec[indices]
comparing_indices = indices - (np.abs(reference_time_vec - left) < np.abs(reference_time_vec - right))

print(reference_time_vec)
print(len(comparing_indices))

In [ ]:
print(np.average(np.abs(ml_mctdh[0:len(comparing_indices),0]-simulation_time_vec[comparing_indices])))

In [ ]:
%matplotlib inline
colors = plt.cm.tab10.colors
colors = list(colors)
colors.append('darkblue')
# A_vals = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0]
# X axis limit
xl = 50
# 'MFSD_SCHW2_ntraj_100_iter_47_A_0.01'
#labels = [f'$A_k$={A_val}' for A_val in A_vals]
plt.rcParams.update({'font.size': 42, 'axes.linewidth': 3, 
                     'xtick.major.width': 3, 'ytick.major.width': 3, 'lines.linewidth': 6.0})
for prop in ['sh_pop_adi']:#,'ave_decoherence_rates','Etot_ave','Ekin_ave','Epot_ave']:
    print('Plotting ', prop)
    plt.figure(figsize=(3.21*4.5, 2.41*4))
    c = 1
    methods = ['FSSH','FSSH2','GFSH']
    for dec_method in ['']:
        for method in methods:
            #plt.subplot(3,3,c)
            #plt.figure(figsize=(3.21*3, 2.41*3))
            print(dec_method, method)
            #for k in range(len(A_vals)):
            file = f'all_methods_dec_wigner_saikat/{method.lower()}_{dec_method}_ntraj_{ntraj}_iter_0_dt_{dt}_istate_{istate}_rep_{rep}_elec_int_{elec_int}/mem_data.hdf'
            F = h5py.File(file)
            if dec_method=='MFSD' and prop=='sh_pop_adi':
                x = np.array(F[f'se_pop_adi/data'])
            else:
                x = np.array(F[f'{prop}/data'])
            time_vec = F['time/data'][:]*units.au2fs/1000
            F.close()
            label = method
            if dec_method!='':
                label += '-'+dec_method
            if prop=='ave_decoherence_rates':
                plt.plot(time_vec, x[:,0,1], label=label)#, color=colors[k])
            elif prop=='sh_pop_adi':
                plt.plot(time_vec, x[:,1], label=label)#, color=colors[k])
            else:
                plt.plot(time_vec, x*units.au2ev*1000, label=label)#, color=colors[k])
    if prop=='sh_pop_adi':
        ml_mctdh = np.loadtxt('reference/ML-MCTDH_n32.dat')
        plt.plot(ml_mctdh[:,0]/1000, ml_mctdh[:,1], label='Ref:ML-MCTDH', ls='dashed', color='black')
        plt.ylabel('S$_1$ SH Population')
    elif prop=='ave_decoherence_rates':
        plt.ylabel('Decoherence rate, 1/a.u.')
    else:
        plt.ylabel('Energy, meV')
            #plt.xlim(0,xl)
            #if c==1:
    plt.legend(ncol=1, loc='upper right')
    plt.xlabel('Time, ps')
    name = dec_method.replace('_REV23','')
            #if method=='FSSH2':
            #    plt.title(f'FSSH-2 with {name}', fontsize=35)
            #elif dec_method=='MFSD':
            #    plt.title(f'MFSD', fontsize=35)
            #else:
            #    plt.title(f'{method} with {name}', fontsize=35)
            #c += 1
    if prop=='sh_pop_adi':
        title = 'Population dynamics, '
    elif prop=='ave_decoherence_rates':
        title = 'Average decoherence rates, '
    elif prop=='Etot_ave':
        title = 'Average total energy, '
    elif prop=='Ekin_ave':
        title = 'Average kinetic energy, '
    elif prop=='Epot_ave':
        title = 'Average potential energy, '
#    plt.suptitle(title+f'dt={dt}a.u.,ntraj={ntraj},elc_int={elec_int} - No SSY', fontsize=30)
    plt.title('No Decoherence Correction, No SSY')
    #for xl in xls:
        #print('Zooming in ', 0, xl)
        #for c3 in range(1,8):
        #    plt.subplot(3,3,c3)
    plt.xlim(0,xl)
    plt.tight_layout()
    plt.savefig(f'no-decoherence_SH_{prop}_no_SSY_xlim_{xl}_ntraj_{ntraj}_dt_{dt}_elec_int_{elec_int}.jpg', dpi=dpi)

In [ ]:
%matplotlib inline
colors = plt.cm.tab10.colors
colors = list(colors)
colors.append('darkblue')
# A_vals = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0]
# X axis limit
xl = 50
# 'MFSD_SCHW2_ntraj_100_iter_47_A_0.01'
#labels = [f'$A_k$={A_val}' for A_val in A_vals]
plt.rcParams.update({'font.size': 42, 'axes.linewidth': 3, 
                     'xtick.major.width': 3, 'ytick.major.width': 3, 'lines.linewidth': 6.0})
for prop in ['sh_pop_adi']:#,'ave_decoherence_rates','Etot_ave','Ekin_ave','Epot_ave']:
    print('Plotting ', prop)
    c = 1
    methods = ['FSSH','FSSH2','GFSH']
    for method in methods:
        plt.figure(figsize=(3.21*4.5, 2.41*4))
        for dec_method in ['','ID-A','BCSH']:
            #plt.subplot(3,1,c)
            #plt.figure(figsize=(3.21*3, 2.41*3))
            print(dec_method, method)
            #for k in range(len(A_vals)):
            file = f'all_methods_dec_wigner_saikat/{method.lower()}_{dec_method}_ntraj_{ntraj}_iter_0_dt_{dt}_istate_{istate}_rep_{rep}_elec_int_{elec_int}/mem_data.hdf'
            F = h5py.File(file)
            if dec_method=='MFSD' and prop=='sh_pop_adi':
                x = np.array(F[f'se_pop_adi/data'])
            else:
                x = np.array(F[f'{prop}/data'])
            time_vec = F['time/data'][:]*units.au2fs/1000
            F.close()
            label = method
            if dec_method!='':
                label = dec_method
#             if dec_method!='':
#                 label += '-'+dec_method
            if prop=='ave_decoherence_rates':
                plt.plot(time_vec, x[:,0,1], label=label)#, color=colors[k])
            elif prop=='sh_pop_adi':
                plt.plot(time_vec, x[:,1], label=label)#, color=colors[k])
            else:
                plt.plot(time_vec, x*units.au2ev*1000, label=label)#, color=colors[k])
        if prop=='sh_pop_adi':
            ml_mctdh = np.loadtxt('reference/ML-MCTDH_n32.dat')
            plt.plot(ml_mctdh[:,0]/1000, ml_mctdh[:,1], label='Ref:ML-MCTDH', ls='dashed', color='black')
            plt.ylabel('S$_1$ SH Population')
        elif prop=='ave_decoherence_rates':
            plt.ylabel('Decoherence rate, 1/a.u.')
        else:
            plt.ylabel('Energy, meV')
                #plt.xlim(0,xl)
                #if c==1:
        plt.legend(ncol=1, loc='upper right')
        plt.xlabel('Time, ps')
        name = dec_method.replace('_REV23','')
                #if method=='FSSH2':
                #    plt.title(f'FSSH-2 with {name}', fontsize=35)
                #elif dec_method=='MFSD':
                #    plt.title(f'MFSD', fontsize=35)
                #else:
                #    plt.title(f'{method} with {name}', fontsize=35)
                #c += 1
        #if prop=='sh_pop_adi':
        title = f'{method.upper()} with ID-A and BCSH, No SSY'
        plt.title(title)
        c += 1
#     elif prop=='ave_decoherence_rates':
#         title = 'Average decoherence rates, '
#     elif prop=='Etot_ave':
#         title = 'Average total energy, '
#     elif prop=='Ekin_ave':
#         title = 'Average kinetic energy, '
#     elif prop=='Epot_ave':
#         title = 'Average potential energy, '
    #plt.suptitle(title+f'dt={dt}a.u.,ntraj={ntraj},elc_int={elec_int} - No SSY', fontsize=30)
    #for xl in xls:
        #print('Zooming in ', 0, xl)
        #for c3 in range(1,8):
        #    plt.subplot(3,3,c3)
        plt.xlim(0,xl)
        plt.tight_layout()
        plt.savefig(f'others_{method}_SH_{prop}_no_SSY_xlim_{xl}_ntraj_{ntraj}_dt_{dt}_elec_int_{elec_int}.jpg', dpi=dpi)

In [ ]:
len(A_vals)


In [ ]:
'fssh'.upper()

In [ ]:
%matplotlib inline
colors = plt.cm.tab10.colors
colors = list(colors)
colors.append('darkblue')
# A_vals = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0]
# X axis limit
xl = 50
# 'MFSD_SCHW2_ntraj_100_iter_47_A_0.01'
# labels = [f'$A_k$={A_val}' for A_val in A_vals]
A_vals = [0.01, 1.0, 100.0, 10000.0, 
          10000000.0, 1000000000.0]
labels = [f'$A_k=10^{{{int(np.log10(A_val))}}}$' for A_val in A_vals]
plt.rcParams.update({'font.size': 42, 'axes.linewidth': 3, 
                     'xtick.major.width': 3, 'ytick.major.width': 3, 'lines.linewidth': 6.0})
for prop in ['sh_pop_adi']:#,'ave_decoherence_rates']: 
    #['sh_pop_adi','ave_decoherence_rates','Etot_ave','Ekin_ave','Epot_ave']:
    print('Plotting ', prop)
#     plt.figure(figsize=(3.21*3*2, 2.41*3))
    methods = ['FSSH','FSSH2','GFSH']
    for method in methods:
        plt.figure(figsize=(3.21*4.5, 2.41*4))
        c = 1
        for dec_method in ['DISH_REV23']:#,'MFSD']:
#         if dec_method=='MFSD':
#             methods = ['FSSH']
#         else:
            #plt.subplot(1,2,c)
            #plt.figure(figsize=(3.21*3, 2.41*3))
            print(dec_method, method)
            for k in range(len(A_vals)):
                file = f'all_methods_dec_wigner_saikat/{method.lower()}_{dec_method}_SCHW1_ntraj_{ntraj}_iter_0_dt_{dt}_A_{A_vals[k]}_istate_{istate}_rep_{rep}_elec_int_{elec_int}/mem_data.hdf'
                F = h5py.File(file)
                if dec_method=='DISH_REV23' and prop=='sh_pop_adi':
                    x = np.array(F[f'se_pop_adi/data'])
                else:
                    x = np.array(F[f'{prop}/data'])
                time_vec = F['time/data'][:]*units.au2fs/1000
                F.close()
                if prop=='ave_decoherence_rates':
                    plt.plot(time_vec, x[:,1,1], label=labels[k], color=colors[k])
                elif prop=='sh_pop_adi':
                    plt.plot(time_vec, x[:,1], label=labels[k], color=colors[k])
                else:
                    plt.plot(time_vec, x*units.au2ev*1000, label=labels[k], color=colors[k])
            if prop=='sh_pop_adi':
                ml_mctdh = np.loadtxt('reference/ML-MCTDH_n32.dat')
                plt.plot(ml_mctdh[:,0]/1000, ml_mctdh[:,1], label='Ref:ML-MCTDH', ls='dashed', color='black')
                plt.ylabel('S$_1$ Quantum Population')
            elif prop=='ave_decoherence_rates':
                plt.ylabel('Decoherence rate, 1/a.u.')
            else:
                plt.ylabel('Energy, meV')
            #plt.xlim(0,xl)
            if c==1:
                plt.legend(ncol=1, fontsize=36, loc='center right')
            plt.xlabel('Time, ps')
            name = dec_method.replace('_REV23','')
#             if method=='FSSH2':
#                 plt.title(f'FSSH-2 with {name}')
#             elif dec_method=='MFSD':
#                 plt.title(f'MFSD', fontsize=35)
#             else:
#                 #plt.title(f'{method} with {name}')
            plt.title(f'{method.upper()}-{name}, Schwartz 1, No SSY')
            c += 1
#     if prop=='sh_pop_adi':
#         title = 'Population dynamics, '
#     elif prop=='ave_decoherence_rates':
#         title = 'Average decoherence rates, '
#     elif prop=='Etot_ave':
#         title = 'Average total energy, '
#     elif prop=='Ekin_ave':
#         title = 'Average kinetic energy, '
#     elif prop=='Epot_ave':
#         title = 'Average potential energy, '
#     #plt.suptitle(title+f'dt={dt}a.u.,ntraj={ntraj},elc_int={elec_int}, Schwartz 1 - No SSY', fontsize=45)
        for xl in xls:
            print('Zooming in ', 0, xl)
            #for c3 in range(1,3):
                #plt.subplot(1,2,c3)
            plt.xlim(0,xl)
            plt.tight_layout()
            plt.savefig(f'{method}_{dec_method}_SE_Schw1_no_SSY_xlim_{xl}_ntraj_{ntraj}_dt_{dt}_elec_int_{elec_int}.jpg', dpi=dpi)

In [ ]:
%matplotlib inline
colors = plt.cm.tab10.colors
colors = list(colors)
colors.append('darkblue')
# A_vals = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0]
# X axis limit
xl = 50
# 'MFSD_SCHW2_ntraj_100_iter_47_A_0.01'
# labels = [f'$A_k$={A_val}' for A_val in A_vals]
A_vals = [0.01, 1.0, 100.0, 10000.0, 
          10000000.0, 1000000000.0]
labels = [f'$A_k=10^{{{int(np.log10(A_val))}}}$' for A_val in A_vals]
plt.rcParams.update({'font.size': 42, 'axes.linewidth': 3, 
                     'xtick.major.width': 3, 'ytick.major.width': 3, 'lines.linewidth': 6.0})
for prop in ['sh_pop_adi']:#,'ave_decoherence_rates']: 
    #['sh_pop_adi','ave_decoherence_rates','Etot_ave','Ekin_ave','Epot_ave']:
    print('Plotting ', prop)
#     plt.figure(figsize=(3.21*3*2, 2.41*3))
    methods = ['FSSH','FSSH2','GFSH']
    for method in methods:
        plt.figure(figsize=(3.21*4.5, 2.41*4))
        c = 1
        for dec_method in ['DISH_REV23']:#,'MFSD']:
#         if dec_method=='MFSD':
#             methods = ['FSSH']
#         else:
            #plt.subplot(1,2,c)
            #plt.figure(figsize=(3.21*3, 2.41*3))
            print(dec_method, method)
            for k in range(len(A_vals)):
                file = f'all_methods_dec_wigner_saikat/{method.lower()}_{dec_method}_SCHW1_ntraj_{ntraj}_iter_0_dt_{dt}_A_{A_vals[k]}_istate_{istate}_rep_{rep}_elec_int_{elec_int}/mem_data.hdf'
                F = h5py.File(file)
                if dec_method=='DISH_REV23' and prop=='sh_pop_adi':
                    x = np.array(F[f'sh_pop_adi/data'])
                else:
                    x = np.array(F[f'{prop}/data'])
                time_vec = F['time/data'][:]*units.au2fs/1000
                F.close()
                if prop=='ave_decoherence_rates':
                    plt.plot(time_vec, x[:,1,1], label=labels[k], color=colors[k])
                elif prop=='sh_pop_adi':
                    plt.plot(time_vec, x[:,1], label=labels[k], color=colors[k])
                else:
                    plt.plot(time_vec, x*units.au2ev*1000, label=labels[k], color=colors[k])
            if prop=='sh_pop_adi':
                ml_mctdh = np.loadtxt('reference/ML-MCTDH_n32.dat')
                plt.plot(ml_mctdh[:,0]/1000, ml_mctdh[:,1], label='Ref:ML-MCTDH', ls='dashed', color='black')
                plt.ylabel('S$_1$ SH Population')
            elif prop=='ave_decoherence_rates':
                plt.ylabel('Decoherence rate, 1/a.u.')
            else:
                plt.ylabel('Energy, meV')
            #plt.xlim(0,xl)
            if c==1:
                plt.legend(ncol=1, fontsize=36, loc='center right')
            plt.xlabel('Time, ps')
            name = dec_method.replace('_REV23','')
#             if method=='FSSH2':
#                 plt.title(f'FSSH-2 with {name}')
#             elif dec_method=='MFSD':
#                 plt.title(f'MFSD', fontsize=35)
#             else:
#                 #plt.title(f'{method} with {name}')
            plt.title(f'{method.upper()}-{name}, Schwartz 1, No SSY')
            c += 1
#     if prop=='sh_pop_adi':
#         title = 'Population dynamics, '
#     elif prop=='ave_decoherence_rates':
#         title = 'Average decoherence rates, '
#     elif prop=='Etot_ave':
#         title = 'Average total energy, '
#     elif prop=='Ekin_ave':
#         title = 'Average kinetic energy, '
#     elif prop=='Epot_ave':
#         title = 'Average potential energy, '
#     #plt.suptitle(title+f'dt={dt}a.u.,ntraj={ntraj},elc_int={elec_int}, Schwartz 1 - No SSY', fontsize=45)
        for xl in xls:
            print('Zooming in ', 0, xl)
            #for c3 in range(1,3):
                #plt.subplot(1,2,c3)
            plt.xlim(0,xl)
            plt.tight_layout()
            plt.savefig(f'{method}_{dec_method}_SH_{prop}_Schw1_no_SSY_xlim_{xl}_ntraj_{ntraj}_dt_{dt}_elec_int_{elec_int}.jpg', dpi=dpi)

In [ ]:
%matplotlib inline
colors = plt.cm.tab10.colors
colors = list(colors)
colors.append('darkblue')
# A_vals = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0]
# X axis limit
xl = 50
# 'MFSD_SCHW2_ntraj_100_iter_47_A_0.01'
# labels = [f'$A_k$={A_val}' for A_val in A_vals]
A_vals = [0.01, 1.0, 100.0, 10000.0, 
          10000000.0, 1000000000.0]
labels = [f'$A_k=10^{{{int(np.log10(A_val))}}}$' for A_val in A_vals]
plt.rcParams.update({'font.size': 42, 'axes.linewidth': 3, 
                     'xtick.major.width': 3, 'ytick.major.width': 3, 'lines.linewidth': 6.0})
for prop in ['sh_pop_adi']:#,'ave_decoherence_rates']: 
    #['sh_pop_adi','ave_decoherence_rates','Etot_ave','Ekin_ave','Epot_ave']:
    print('Plotting ', prop)
#     plt.figure(figsize=(3.21*3*2, 2.41*3))
    methods = ['FSSH','FSSH2','GFSH']
    for method in methods:
        plt.figure(figsize=(3.21*4.5, 2.41*4))
        c = 1
        for dec_method in ['DISH_REV23']:#,'MFSD']:
#         if dec_method=='MFSD':
#             methods = ['FSSH']
#         else:
            #plt.subplot(1,2,c)
            #plt.figure(figsize=(3.21*3, 2.41*3))
            print(dec_method, method)
            for k in range(len(A_vals)):
                file = f'all_methods_dec_wigner_saikat/{method.lower()}_{dec_method}_SCHW1_ntraj_{ntraj}_iter_0_dt_{dt}_A_{A_vals[k]}_istate_{istate}_rep_{rep}_elec_int_{elec_int}/mem_data.hdf'
                F = h5py.File(file)
                #print(F.keys())
                if dec_method=='DISH_REV23' and prop=='sh_pop_adi':
                    x = np.array(F[f'sh_pop_adi/data'])
                else:
                    x = np.array(F[f'{prop}/data'])
                time_vec = F['time/data'][:]*units.au2fs/1000
                F.close()
                if prop=='ave_decoherence_rates':
                    plt.plot(time_vec, x[:,1,1], label=labels[k], color=colors[k])
                elif prop=='sh_pop_adi':
                    plt.plot(time_vec, x[:,1], label=labels[k], color=colors[k])
                else:
                    plt.plot(time_vec, x*units.au2ev*1000, label=labels[k], color=colors[k])
            if prop=='sh_pop_adi':
                ml_mctdh = np.loadtxt('reference/ML-MCTDH_n32.dat')
                plt.plot(ml_mctdh[:,0]/1000, ml_mctdh[:,1], label='Ref:ML-MCTDH', ls='dashed', color='black')
                plt.ylabel('S$_1$ SH Population')
            elif prop=='ave_decoherence_rates':
                plt.ylabel('Decoherence rate, 1/a.u.')
            else:
                plt.ylabel('Energy, meV')
            #plt.xlim(0,xl)
            if c==1:
                plt.legend(ncol=1, fontsize=36, loc='center right')
            plt.xlabel('Time, ps')
            name = dec_method.replace('_REV23','')
#             if method=='FSSH2':
#                 plt.title(f'FSSH-2 with {name}')
#             elif dec_method=='MFSD':
#                 plt.title(f'MFSD', fontsize=35)
#             else:
#                 #plt.title(f'{method} with {name}')
            plt.title(f'{method.upper()}-{name}, Schwartz 1, No SSY')
            c += 1
#     if prop=='sh_pop_adi':
#         title = 'Population dynamics, '
#     elif prop=='ave_decoherence_rates':
#         title = 'Average decoherence rates, '
#     elif prop=='Etot_ave':
#         title = 'Average total energy, '
#     elif prop=='Ekin_ave':
#         title = 'Average kinetic energy, '
#     elif prop=='Epot_ave':
#         title = 'Average potential energy, '
#     #plt.suptitle(title+f'dt={dt}a.u.,ntraj={ntraj},elc_int={elec_int}, Schwartz 1 - No SSY', fontsize=45)
        for xl in xls:
            print('Zooming in ', 0, xl)
            #for c3 in range(1,3):
                #plt.subplot(1,2,c3)
            plt.xlim(0,xl)
            plt.tight_layout()
            plt.savefig(f'{method}_{dec_method}__{prop}_Schw1_no_SSY_xlim_{xl}_ntraj_{ntraj}_dt_{dt}_elec_int_{elec_int}.jpg', dpi=dpi)

# Schwartz 2

In [ ]:
%matplotlib inline
colors = plt.cm.tab10.colors
colors = list(colors)
colors.append('darkblue')
# A_vals = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0]
# X axis limit
xl = 50
# 'MFSD_SCHW2_ntraj_100_iter_47_A_0.01'
# labels = [f'$A_k$={A_val}' for A_val in A_vals]
A_vals = [0.01, 1.0, 100.0, 10000.0, 
          10000000.0, 1000000000.0]
labels = [f'$A_k=10^{{{int(np.log10(A_val))}}}$' for A_val in A_vals]
plt.rcParams.update({'font.size': 42, 'axes.linewidth': 3, 
                     'xtick.major.width': 3, 'ytick.major.width': 3, 'lines.linewidth': 6.0})
for prop in ['sh_pop_adi']:#,'ave_decoherence_rates']: 
    #['sh_pop_adi','ave_decoherence_rates','Etot_ave','Ekin_ave','Epot_ave']:
    print('Plotting ', prop)
#     plt.figure(figsize=(3.21*3*2, 2.41*3))
    methods = ['FSSH','FSSH2','GFSH']
    for method in methods:
        plt.figure(figsize=(3.21*4.5, 2.41*4))
        c = 1
        for dec_method in ['DISH_REV23']:#,'MFSD']:
#         if dec_method=='MFSD':
#             methods = ['FSSH']
#         else:
#             plt.subplot(1,2,c)
            #plt.figure(figsize=(3.21*3, 2.41*3))
            print(dec_method, method)
            for k in range(len(A_vals)):
                file = f'all_methods_dec_wigner_saikat/{method.lower()}_{dec_method}_SCHW2_ntraj_{ntraj}_iter_0_dt_{dt}_A_{A_vals[k]}_istate_{istate}_rep_{rep}_elec_int_{elec_int}/mem_data.hdf'
                F = h5py.File(file)
                if dec_method=='DISH_REV23' and prop=='sh_pop_adi':
                    x = np.array(F[f'se_pop_adi/data'])
                else:
                    x = np.array(F[f'{prop}/data'])
                time_vec = F['time/data'][:]*units.au2fs/1000
                F.close()
                if prop=='ave_decoherence_rates':
                    plt.plot(time_vec, x[:,1,1], label=labels[k], color=colors[k])
                elif prop=='sh_pop_adi':
                    plt.plot(time_vec, x[:,1], label=labels[k], color=colors[k])
                else:
                    plt.plot(time_vec, x*units.au2ev*1000, label=labels[k], color=colors[k])
            if prop=='sh_pop_adi':
                ml_mctdh = np.loadtxt('reference/ML-MCTDH_n32.dat')
                plt.plot(ml_mctdh[:,0]/1000, ml_mctdh[:,1], label='Ref:ML-MCTDH', ls='dashed', color='black')
                plt.ylabel('S$_1$ Quantum Population')
            elif prop=='ave_decoherence_rates':
                plt.ylabel('Decoherence rate, 1/a.u.')
            else:
                plt.ylabel('Energy, meV')
            #plt.xlim(0,xl)
            if c==1:
                plt.legend(ncol=1, fontsize=36, loc='center right')
            plt.xlabel('Time, ps')
            name = dec_method.replace('_REV23','')
#             if method=='FSSH2':
#                 plt.title(f'FSSH-2 with {name}')
#             elif dec_method=='MFSD':
#                 plt.title(f'MFSD', fontsize=35)
#             else:
#                 #plt.title(f'{method} with {name}')
            plt.title(f'{method.upper()}-{name}, Schwartz 2, No SSY')
            c += 1
#     if prop=='sh_pop_adi':
#         title = 'Population dynamics, '
#     elif prop=='ave_decoherence_rates':
#         title = 'Average decoherence rates, '
#     elif prop=='Etot_ave':
#         title = 'Average total energy, '
#     elif prop=='Ekin_ave':
#         title = 'Average kinetic energy, '
#     elif prop=='Epot_ave':
#         title = 'Average potential energy, '
    #plt.suptitle(title+f'dt={dt}a.u.,ntraj={ntraj},elc_int={elec_int}, Schwartz 1 - No SSY', fontsize=45)
        for xl in xls:
            print('Zooming in ', 0, xl)
            #for c3 in range(1,3):
            #    plt.subplot(1,2,c3)
            plt.xlim(0,xl)
            plt.tight_layout()
            plt.savefig(f'{method}_{dec_method}_SE_{prop}_Schw2_no_SSY_xlim_{xl}_ntraj_{ntraj}_dt_{dt}_elec_int_{elec_int}.jpg', dpi=dpi)

In [ ]:
%matplotlib inline
colors = plt.cm.tab10.colors
colors = list(colors)
colors.append('darkblue')
# A_vals = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0]
# X axis limit
xl = 50
# 'MFSD_SCHW2_ntraj_100_iter_47_A_0.01'
# labels = [f'$A_k$={A_val}' for A_val in A_vals]
A_vals = [0.01, 1.0, 100.0, 10000.0, 
          10000000.0, 1000000000.0]
labels = [f'$A_k=10^{{{int(np.log10(A_val))}}}$' for A_val in A_vals]
plt.rcParams.update({'font.size': 42, 'axes.linewidth': 3, 
                     'xtick.major.width': 3, 'ytick.major.width': 3, 'lines.linewidth': 6.0})
for prop in ['sh_pop_adi']:#,'ave_decoherence_rates']: 
    #['sh_pop_adi','ave_decoherence_rates','Etot_ave','Ekin_ave','Epot_ave']:
    print('Plotting ', prop)
#     plt.figure(figsize=(3.21*3*2, 2.41*3))
    methods = ['FSSH','FSSH2','GFSH']
    for method in methods:
        plt.figure(figsize=(3.21*4.5, 2.41*4))
        c = 1
        for dec_method in ['DISH_REV23']:#,'MFSD']:
#         if dec_method=='MFSD':
#             methods = ['FSSH']
#         else:
#             plt.subplot(1,2,c)
            #plt.figure(figsize=(3.21*3, 2.41*3))
            print(dec_method, method)
            for k in range(len(A_vals)):
                file = f'all_methods_dec_wigner_saikat/{method.lower()}_{dec_method}_SCHW2_ntraj_{ntraj}_iter_0_dt_{dt}_A_{A_vals[k]}_istate_{istate}_rep_{rep}_elec_int_{elec_int}/mem_data.hdf'
                F = h5py.File(file)
                if dec_method=='DISH_REV23' and prop=='sh_pop_adi':
                    x = np.array(F[f'sh_pop_adi/data'])
                else:
                    x = np.array(F[f'{prop}/data'])
                time_vec = F['time/data'][:]*units.au2fs/1000
                F.close()
                if prop=='ave_decoherence_rates':
                    plt.plot(time_vec, x[:,1,1], label=labels[k], color=colors[k])
                elif prop=='sh_pop_adi':
                    plt.plot(time_vec, x[:,1], label=labels[k], color=colors[k])
                else:
                    plt.plot(time_vec, x*units.au2ev*1000, label=labels[k], color=colors[k])
            if prop=='sh_pop_adi':
                ml_mctdh = np.loadtxt('reference/ML-MCTDH_n32.dat')
                plt.plot(ml_mctdh[:,0]/1000, ml_mctdh[:,1], label='Ref:ML-MCTDH', ls='dashed', color='black')
                plt.ylabel('S$_1$ SH Population')
            elif prop=='ave_decoherence_rates':
                plt.ylabel('Decoherence rate, 1/a.u.')
            else:
                plt.ylabel('Energy, meV')
            #plt.xlim(0,xl)
            if c==1:
                plt.legend(ncol=1, fontsize=36, loc='center right')
            plt.xlabel('Time, ps')
            name = dec_method.replace('_REV23','')
#             if method=='FSSH2':
#                 plt.title(f'FSSH-2 with {name}')
#             elif dec_method=='MFSD':
#                 plt.title(f'MFSD', fontsize=35)
#             else:
#                 #plt.title(f'{method} with {name}')
            plt.title(f'{method.upper()}-{name}, Schwartz 2, No SSY')
            c += 1
#     if prop=='sh_pop_adi':
#         title = 'Population dynamics, '
#     elif prop=='ave_decoherence_rates':
#         title = 'Average decoherence rates, '
#     elif prop=='Etot_ave':
#         title = 'Average total energy, '
#     elif prop=='Ekin_ave':
#         title = 'Average kinetic energy, '
#     elif prop=='Epot_ave':
#         title = 'Average potential energy, '
    #plt.suptitle(title+f'dt={dt}a.u.,ntraj={ntraj},elc_int={elec_int}, Schwartz 1 - No SSY', fontsize=45)
        for xl in xls:
            print('Zooming in ', 0, xl)
            #for c3 in range(1,3):
            #    plt.subplot(1,2,c3)
            plt.xlim(0,xl)
            plt.tight_layout()
            plt.savefig(f'{method}_{dec_method}_SH_{prop}_Schw2_no_SSY_xlim_{xl}_ntraj_{ntraj}_dt_{dt}_elec_int_{elec_int}.jpg', dpi=dpi)

In [ ]:
%matplotlib inline
colors = plt.cm.tab10.colors
colors = list(colors)
colors.append('darkblue')
# A_vals = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0]
# X axis limit
xl = 50
# 'MFSD_SCHW2_ntraj_100_iter_47_A_0.01'
# labels = [f'$A_k$={A_val}' for A_val in A_vals]
A_vals = [0.01, 1.0, 100.0, 10000.0, 
          10000000.0, 1000000000.0]
labels = [f'$A_k=10^{{{int(np.log10(A_val))}}}$' for A_val in A_vals]
plt.rcParams.update({'font.size': 42, 'axes.linewidth': 3, 
                     'xtick.major.width': 3, 'ytick.major.width': 3, 'lines.linewidth': 6.0})
for prop in ['sh_pop_adi']:#,'ave_decoherence_rates']: 
    #['sh_pop_adi','ave_decoherence_rates','Etot_ave','Ekin_ave','Epot_ave']:
    print('Plotting ', prop)
#     plt.figure(figsize=(3.21*3*2, 2.41*3))
    methods = ['FSSH','FSSH2','GFSH']
    for method in methods:
        plt.figure(figsize=(3.21*4.5, 2.41*4))
        c = 1
        for dec_method in ['SDM']:#,'MFSD']:
#         if dec_method=='MFSD':
#             methods = ['FSSH']
#         else:
#             plt.subplot(1,2,c)
            #plt.figure(figsize=(3.21*3, 2.41*3))
            print(dec_method, method)
            for k in range(len(A_vals)):
                file = f'all_methods_dec_wigner_saikat/{method.lower()}_{dec_method}_SCHW2_ntraj_{ntraj}_iter_0_dt_{dt}_A_{A_vals[k]}_istate_{istate}_rep_{rep}_elec_int_{elec_int}/mem_data.hdf'
                F = h5py.File(file)
                if dec_method=='DISH_REV23' and prop=='sh_pop_adi':
                    x = np.array(F[f'se_pop_adi/data'])
                else:
                    x = np.array(F[f'{prop}/data'])
                time_vec = F['time/data'][:]*units.au2fs/1000
                F.close()
                if prop=='ave_decoherence_rates':
                    plt.plot(time_vec, x[:,1,1], label=labels[k], color=colors[k])
                elif prop=='sh_pop_adi':
                    plt.plot(time_vec, x[:,1], label=labels[k], color=colors[k])
                else:
                    plt.plot(time_vec, x*units.au2ev*1000, label=labels[k], color=colors[k])
            if prop=='sh_pop_adi':
                ml_mctdh = np.loadtxt('reference/ML-MCTDH_n32.dat')
                plt.plot(ml_mctdh[:,0]/1000, ml_mctdh[:,1], label='Ref:ML-MCTDH', ls='dashed', color='black')
                plt.ylabel('S$_1$ SH Population')
            elif prop=='ave_decoherence_rates':
                plt.ylabel('Decoherence rate, 1/a.u.')
            else:
                plt.ylabel('Energy, meV')
            #plt.xlim(0,xl)
            if c==1:
                plt.legend(ncol=1, fontsize=36, loc='center right')
            plt.xlabel('Time, ps')
            name = dec_method.replace('_REV23','')
#             if method=='FSSH2':
#                 plt.title(f'FSSH-2 with {name}')
#             elif dec_method=='MFSD':
#                 plt.title(f'MFSD', fontsize=35)
#             else:
#                 #plt.title(f'{method} with {name}')
            plt.title(f'{method.upper()}-{name}, Schwartz 2, No SSY')
            c += 1
#     if prop=='sh_pop_adi':
#         title = 'Population dynamics, '
#     elif prop=='ave_decoherence_rates':
#         title = 'Average decoherence rates, '
#     elif prop=='Etot_ave':
#         title = 'Average total energy, '
#     elif prop=='Ekin_ave':
#         title = 'Average kinetic energy, '
#     elif prop=='Epot_ave':
#         title = 'Average potential energy, '
    #plt.suptitle(title+f'dt={dt}a.u.,ntraj={ntraj},elc_int={elec_int}, Schwartz 1 - No SSY', fontsize=45)
        for xl in xls:
            print('Zooming in ', 0, xl)
            #for c3 in range(1,3):
            #    plt.subplot(1,2,c3)
            plt.xlim(0,xl)
            plt.tight_layout()
            plt.savefig(f'{method}_{dec_method}_SH_{prop}_Schw2_no_SSY_xlim_{xl}_ntraj_{ntraj}_dt_{dt}_elec_int_{elec_int}.jpg', dpi=dpi)

In [ ]:
%matplotlib inline
colors = plt.cm.tab10.colors
colors = list(colors)
colors.append('darkblue')
# A_vals = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0]
# X axis limit
xl = 50
# 'MFSD_SCHW2_ntraj_100_iter_47_A_0.01'
# labels = [f'$A_k$={A_val}' for A_val in A_vals]
A_vals = [0.01, 1.0, 100.0, 10000.0, 
          10000000.0, 1000000000.0]
labels = [f'$A_k=10^{{{int(np.log10(A_val))}}}$' for A_val in A_vals]
plt.rcParams.update({'font.size': 42, 'axes.linewidth': 3, 
                     'xtick.major.width': 3, 'ytick.major.width': 3, 'lines.linewidth': 6.0})
for prop in ['sh_pop_adi']:#,'ave_decoherence_rates']: 
    #['sh_pop_adi','ave_decoherence_rates','Etot_ave','Ekin_ave','Epot_ave']:
    print('Plotting ', prop)
#     plt.figure(figsize=(3.21*3*2, 2.41*3))
    methods = ['FSSH','FSSH2','GFSH']
    for method in methods:
        plt.figure(figsize=(3.21*4.5, 2.41*4))
        c = 1
        for dec_method in ['SDM']:#,'MFSD']:
#         if dec_method=='MFSD':
#             methods = ['FSSH']
#         else:
#             plt.subplot(1,2,c)
            #plt.figure(figsize=(3.21*3, 2.41*3))
            print(dec_method, method)
            for k in range(len(A_vals)):
                file = f'all_methods_dec_wigner_saikat/{method.lower()}_{dec_method}_SCHW2_ntraj_{ntraj}_iter_0_dt_{dt}_A_{A_vals[k]}_istate_{istate}_rep_{rep}_elec_int_{elec_int}/mem_data.hdf'
                F = h5py.File(file)
                if dec_method=='DISH_REV23' and prop=='sh_pop_adi':
                    x = np.array(F[f'se_pop_adi/data'])
                else:
                    x = np.array(F[f'se_pop_adi/data'])
                    #x = np.array(F[f'{prop}/data'])
                time_vec = F['time/data'][:]*units.au2fs/1000
                F.close()
                if prop=='ave_decoherence_rates':
                    plt.plot(time_vec, x[:,1,1], label=labels[k], color=colors[k])
                elif prop=='sh_pop_adi':
                    plt.plot(time_vec, x[:,1], label=labels[k], color=colors[k])
                else:
                    plt.plot(time_vec, x*units.au2ev*1000, label=labels[k], color=colors[k])
            if prop=='sh_pop_adi':
                ml_mctdh = np.loadtxt('reference/ML-MCTDH_n32.dat')
                plt.plot(ml_mctdh[:,0]/1000, ml_mctdh[:,1], label='Ref:ML-MCTDH', ls='dashed', color='black')
                plt.ylabel('S$_1$ Quantum Population')
            elif prop=='ave_decoherence_rates':
                plt.ylabel('Decoherence rate, 1/a.u.')
            else:
                plt.ylabel('Energy, meV')
            #plt.xlim(0,xl)
            if c==1:
                plt.legend(ncol=1, fontsize=36, loc='center right')
            plt.xlabel('Time, ps')
            name = dec_method.replace('_REV23','')
#             if method=='FSSH2':
#                 plt.title(f'FSSH-2 with {name}')
#             elif dec_method=='MFSD':
#                 plt.title(f'MFSD', fontsize=35)
#             else:
#                 #plt.title(f'{method} with {name}')
            plt.title(f'{method.upper()}-{name}, Schwartz 2, No SSY')
            c += 1
#     if prop=='sh_pop_adi':
#         title = 'Population dynamics, '
#     elif prop=='ave_decoherence_rates':
#         title = 'Average decoherence rates, '
#     elif prop=='Etot_ave':
#         title = 'Average total energy, '
#     elif prop=='Ekin_ave':
#         title = 'Average kinetic energy, '
#     elif prop=='Epot_ave':
#         title = 'Average potential energy, '
    #plt.suptitle(title+f'dt={dt}a.u.,ntraj={ntraj},elc_int={elec_int}, Schwartz 1 - No SSY', fontsize=45)
        for xl in xls:
            print('Zooming in ', 0, xl)
            #for c3 in range(1,3):
            #    plt.subplot(1,2,c3)
            plt.xlim(0,xl)
            plt.tight_layout()
            plt.savefig(f'{method}_{dec_method}_SE_{prop}_Schw2_no_SSY_xlim_{xl}_ntraj_{ntraj}_dt_{dt}_elec_int_{elec_int}.jpg', dpi=dpi)

## EDC

In [ ]:
%matplotlib inline
colors = plt.cm.tab10.colors
colors = list(colors)
colors.append('darkblue')
# X axis limit
xl = 50
#eps_vals = [0.01, 0.05, 0.1, 0.2, 0.4, 1.0, 5.0, 10.0, 20.0, 40.0, 80.0]
eps_vals = [0.01, 0.1, 1.0, 10.0, 100.0, 1000.0] 
#, 0.4, 1.0, 5.0, 10.0, 20.0, 40.0, 80.0]
labels = [f'$\\epsilon$={eps_val}' for eps_val in eps_vals]
plt.rcParams.update({'font.size': 42, 'axes.linewidth': 3, 
                     'xtick.major.width': 3, 'ytick.major.width': 3, 'lines.linewidth': 6.0})
for prop in ['sh_pop_adi']:#,'ave_decoherence_rates','Etot_ave','Ekin_ave','Epot_ave']:
    print('Plotting ', prop)
    methods = ['FSSH','FSSH2','GFSH']
    for method in methods:
        plt.figure(figsize=(3.21*4.5, 2.41*4))
        c = 1
        for dec_method in ['DISH_REV23']:#,'MFSD']:
#         if dec_method=='MFSD':
#             methods = ['FSSH']
#         else:
#             plt.subplot(1,2,c)
            #plt.figure(figsize=(3.21*3, 2.41*3))
            print(dec_method, method)
            for k in range(len(eps_vals)):
                file = f'all_methods_dec_wigner_saikat/{method.lower()}_{dec_method}_EDC_ntraj_{ntraj}_iter_0_dt_{dt}_eps_param_{eps_vals[k]}_istate_{istate}_rep_{rep}_elec_int_{elec_int}/mem_data.hdf'
                F = h5py.File(file)
                if dec_method=='DISH_REV23' and prop=='sh_pop_adi':
                    x = np.array(F[f'se_pop_adi/data'])
                else:
                    x = np.array(F[f'{prop}/data'])
                time_vec = F['time/data'][:]*units.au2fs/1000
                F.close()
                if prop=='ave_decoherence_rates':
                    plt.plot(time_vec, x[:,0,1], label=labels[k], color=colors[k])
                elif prop=='sh_pop_adi':
                    plt.plot(time_vec, x[:,1], label=labels[k], color=colors[k])
                else:
                    plt.plot(time_vec, x*units.au2ev*1000, label=labels[k], color=colors[k])
            if prop=='sh_pop_adi':
                ml_mctdh = np.loadtxt('reference/ML-MCTDH_n32.dat')
                plt.plot(ml_mctdh[:,0]/1000, ml_mctdh[:,1], label='Ref:ML-MCTDH', ls='dashed', color='black')
                plt.ylabel('S$_1$ Quantum Population')
            elif prop=='ave_decoherence_rates':
                plt.ylabel('Decoherence rate, 1/a.u.')
            else:
                plt.ylabel('Energy, meV')
            plt.xlim(0,xl)
            if c==1:
                plt.legend(fontsize=36, ncol=1, loc='center right')
            plt.xlabel('Time, ps')
            name = dec_method.replace('_REV23','')
#             if method=='FSSH2':
#                 plt.title(f'FSSH-2 with {name}', fontsize=35)
#             elif dec_method=='MFSD':
#                 plt.title(f'MFSD', fontsize=35)
#             else:
#                 plt.title(f'{method} with {name}', fontsize=35)
            plt.title(f'{method.upper()}-{name}, EDC, No SSY')
            c += 1
#         if prop=='sh_pop_adi':
#             title = 'Population dynamics, '
#         elif prop=='ave_decoherence_rates':
#             title = 'Average decoherence rates, '
#         elif prop=='Etot_ave':
#             title = 'Average total energy, '
#         elif prop=='Ekin_ave':
#             title = 'Average kinetic energy, '
#         elif prop=='Epot_ave':
#             title = 'Average potential energy, '
        #plt.suptitle(title+f'dt={dt}a.u.,ntraj={ntraj},elc_int={elec_int}, EDC - No SSY', fontsize=45)
        for xl in xls:
            print('Zooming in ', 0, xl)
#             for c3 in range(1,3):
#                 plt.subplot(1,2,c3)
            plt.xlim(0,xl)
            plt.tight_layout()
            plt.savefig(f'{method}_{dec_method}_SE_{prop}_EDC_no_SSY_xlim_{xl}_ntraj_{ntraj}_dt_{dt}_elec_int_{elec_int}.jpg', dpi=dpi) 

In [ ]:
%matplotlib inline
colors = plt.cm.tab10.colors
colors = list(colors)
colors.append('darkblue')
# X axis limit
xl = 50
#eps_vals = [0.01, 0.05, 0.1, 0.2, 0.4, 1.0, 5.0, 10.0, 20.0, 40.0, 80.0]
eps_vals = [0.01, 0.1, 1.0, 10.0, 100.0, 1000.0] 
#, 0.4, 1.0, 5.0, 10.0, 20.0, 40.0, 80.0]
labels = [f'$\\epsilon$={eps_val}' for eps_val in eps_vals]
plt.rcParams.update({'font.size': 42, 'axes.linewidth': 3, 
                     'xtick.major.width': 3, 'ytick.major.width': 3, 'lines.linewidth': 6.0})
for prop in ['sh_pop_adi']:#,'ave_decoherence_rates','Etot_ave','Ekin_ave','Epot_ave']:
    print('Plotting ', prop)
    methods = ['FSSH','FSSH2','GFSH']
    for method in methods:
        plt.figure(figsize=(3.21*4.5, 2.41*4))
        c = 1
        for dec_method in ['DISH_REV23']:#,'MFSD']:
#         if dec_method=='MFSD':
#             methods = ['FSSH']
#         else:
#             plt.subplot(1,2,c)
            #plt.figure(figsize=(3.21*3, 2.41*3))
            print(dec_method, method)
            for k in range(len(eps_vals)):
                file = f'all_methods_dec_wigner_saikat/{method.lower()}_{dec_method}_EDC_ntraj_{ntraj}_iter_0_dt_{dt}_eps_param_{eps_vals[k]}_istate_{istate}_rep_{rep}_elec_int_{elec_int}/mem_data.hdf'
                F = h5py.File(file)
                if dec_method=='DISH_REV23' and prop=='sh_pop_adi':
                    x = np.array(F[f'sh_pop_adi/data'])
                else:
                    x = np.array(F[f'{prop}/data'])
                time_vec = F['time/data'][:]*units.au2fs/1000
                F.close()
                if prop=='ave_decoherence_rates':
                    plt.plot(time_vec, x[:,0,1], label=labels[k], color=colors[k])
                elif prop=='sh_pop_adi':
                    plt.plot(time_vec, x[:,1], label=labels[k], color=colors[k])
                else:
                    plt.plot(time_vec, x*units.au2ev*1000, label=labels[k], color=colors[k])
            if prop=='sh_pop_adi':
                ml_mctdh = np.loadtxt('reference/ML-MCTDH_n32.dat')
                plt.plot(ml_mctdh[:,0]/1000, ml_mctdh[:,1], label='Ref:ML-MCTDH', ls='dashed', color='black')
                plt.ylabel('S$_1$ SH Population')
            elif prop=='ave_decoherence_rates':
                plt.ylabel('Decoherence rate, 1/a.u.')
            else:
                plt.ylabel('Energy, meV')
            plt.xlim(0,xl)
            if c==1:
                plt.legend(fontsize=36, ncol=1, loc='center right')
            plt.xlabel('Time, ps')
            name = dec_method.replace('_REV23','')
#             if method=='FSSH2':
#                 plt.title(f'FSSH-2 with {name}', fontsize=35)
#             elif dec_method=='MFSD':
#                 plt.title(f'MFSD', fontsize=35)
#             else:
#                 plt.title(f'{method} with {name}', fontsize=35)
            plt.title(f'{method.upper()}-{name}, EDC, No SSY')
            c += 1
#         if prop=='sh_pop_adi':
#             title = 'Population dynamics, '
#         elif prop=='ave_decoherence_rates':
#             title = 'Average decoherence rates, '
#         elif prop=='Etot_ave':
#             title = 'Average total energy, '
#         elif prop=='Ekin_ave':
#             title = 'Average kinetic energy, '
#         elif prop=='Epot_ave':
#             title = 'Average potential energy, '
        #plt.suptitle(title+f'dt={dt}a.u.,ntraj={ntraj},elc_int={elec_int}, EDC - No SSY', fontsize=45)
        for xl in xls:
            print('Zooming in ', 0, xl)
#             for c3 in range(1,3):
#                 plt.subplot(1,2,c3)
            plt.xlim(0,xl)
            plt.tight_layout()
            plt.savefig(f'{method}_{dec_method}_SH_{prop}_EDC_no_SSY_xlim_{xl}_ntraj_{ntraj}_dt_{dt}_elec_int_{elec_int}.jpg', dpi=dpi) 

In [ ]:
%matplotlib inline
colors = plt.cm.tab10.colors
colors = list(colors)
colors.append('darkblue')
# X axis limit
xl = 50
#eps_vals = [0.01, 0.05, 0.1, 0.2, 0.4, 1.0, 5.0, 10.0, 20.0, 40.0, 80.0]
eps_vals = [0.01, 0.1, 1.0, 10.0, 100.0, 1000.0] 
#, 0.4, 1.0, 5.0, 10.0, 20.0, 40.0, 80.0]
labels = [f'$\\epsilon$={eps_val}' for eps_val in eps_vals]
plt.rcParams.update({'font.size': 42, 'axes.linewidth': 3, 
                     'xtick.major.width': 3, 'ytick.major.width': 3, 'lines.linewidth': 6.0})
for prop in ['sh_pop_adi']:#,'ave_decoherence_rates','Etot_ave','Ekin_ave','Epot_ave']:
    print('Plotting ', prop)
    methods = ['FSSH','FSSH2','GFSH']
    for method in methods:
        plt.figure(figsize=(3.21*4.5, 2.41*4))
        c = 1
        for dec_method in ['SDM']:#,'MFSD']:
#         if dec_method=='MFSD':
#             methods = ['FSSH']
#         else:
#             plt.subplot(1,2,c)
            #plt.figure(figsize=(3.21*3, 2.41*3))
            print(dec_method, method)
            for k in range(len(eps_vals)):
                file = f'all_methods_dec_wigner_saikat/{method.lower()}_{dec_method}_EDC_ntraj_{ntraj}_iter_0_dt_{dt}_eps_param_{eps_vals[k]}_istate_{istate}_rep_{rep}_elec_int_{elec_int}/mem_data.hdf'
                F = h5py.File(file)
                if dec_method=='DISH_REV23' and prop=='sh_pop_adi':
                    x = np.array(F[f'se_pop_adi/data'])
                else:
                    x = np.array(F[f'{prop}/data'])
                time_vec = F['time/data'][:]*units.au2fs/1000
                F.close()
                if prop=='ave_decoherence_rates':
                    plt.plot(time_vec, x[:,0,1], label=labels[k], color=colors[k])
                elif prop=='sh_pop_adi':
                    plt.plot(time_vec, x[:,1], label=labels[k], color=colors[k])
                else:
                    plt.plot(time_vec, x*units.au2ev*1000, label=labels[k], color=colors[k])
            if prop=='sh_pop_adi':
                ml_mctdh = np.loadtxt('reference/ML-MCTDH_n32.dat')
                plt.plot(ml_mctdh[:,0]/1000, ml_mctdh[:,1], label='Ref:ML-MCTDH', ls='dashed', color='black')
                plt.ylabel('S$_1$ SH Population')
            elif prop=='ave_decoherence_rates':
                plt.ylabel('Decoherence rate, 1/a.u.')
            else:
                plt.ylabel('Energy, meV')
            plt.xlim(0,xl)
            if c==1:
                plt.legend(fontsize=36, ncol=1, loc='center right')
            plt.xlabel('Time, ps')
            name = dec_method.replace('_REV23','')
#             if method=='FSSH2':
#                 plt.title(f'FSSH-2 with {name}', fontsize=35)
#             elif dec_method=='MFSD':
#                 plt.title(f'MFSD', fontsize=35)
#             else:
#                 plt.title(f'{method} with {name}', fontsize=35)
            plt.title(f'{method.upper()}-{name}, EDC, No SSY')
            c += 1
#         if prop=='sh_pop_adi':
#             title = 'Population dynamics, '
#         elif prop=='ave_decoherence_rates':
#             title = 'Average decoherence rates, '
#         elif prop=='Etot_ave':
#             title = 'Average total energy, '
#         elif prop=='Ekin_ave':
#             title = 'Average kinetic energy, '
#         elif prop=='Epot_ave':
#             title = 'Average potential energy, '
        #plt.suptitle(title+f'dt={dt}a.u.,ntraj={ntraj},elc_int={elec_int}, EDC - No SSY', fontsize=45)
        for xl in xls:
            print('Zooming in ', 0, xl)
#             for c3 in range(1,3):
#                 plt.subplot(1,2,c3)
            plt.xlim(0,xl)
            plt.tight_layout()
            plt.savefig(f'{method}_{dec_method}_SH_{prop}_EDC_no_SSY_xlim_{xl}_ntraj_{ntraj}_dt_{dt}_elec_int_{elec_int}.jpg', dpi=dpi) 

In [ ]:
%matplotlib inline
colors = plt.cm.tab10.colors
colors = list(colors)
colors.append('darkblue')
# X axis limit
xl = 50
#eps_vals = [0.01, 0.05, 0.1, 0.2, 0.4, 1.0, 5.0, 10.0, 20.0, 40.0, 80.0]
eps_vals = [0.01, 0.1, 1.0, 10.0, 100.0, 1000.0] 
#, 0.4, 1.0, 5.0, 10.0, 20.0, 40.0, 80.0]
labels = [f'$\\epsilon$={eps_val}' for eps_val in eps_vals]
plt.rcParams.update({'font.size': 42, 'axes.linewidth': 3, 
                     'xtick.major.width': 3, 'ytick.major.width': 3, 'lines.linewidth': 6.0})
for prop in ['sh_pop_adi']:#,'ave_decoherence_rates','Etot_ave','Ekin_ave','Epot_ave']:
    print('Plotting ', prop)
    methods = ['FSSH','FSSH2','GFSH']
    for method in methods:
        plt.figure(figsize=(3.21*4.5, 2.41*4))
        c = 1
        for dec_method in ['SDM']:#,'MFSD']:
#         if dec_method=='MFSD':
#             methods = ['FSSH']
#         else:
#             plt.subplot(1,2,c)
            #plt.figure(figsize=(3.21*3, 2.41*3))
            print(dec_method, method)
            for k in range(len(eps_vals)):
                file = f'all_methods_dec_wigner_saikat/{method.lower()}_{dec_method}_EDC_ntraj_{ntraj}_iter_0_dt_{dt}_eps_param_{eps_vals[k]}_istate_{istate}_rep_{rep}_elec_int_{elec_int}/mem_data.hdf'
                F = h5py.File(file)
                if dec_method=='DISH_REV23' and prop=='sh_pop_adi':
                    x = np.array(F[f'se_pop_adi/data'])
                else:
                    x = np.array(F[f'se_pop_adi/data'])
                time_vec = F['time/data'][:]*units.au2fs/1000
                F.close()
                if prop=='ave_decoherence_rates':
                    plt.plot(time_vec, x[:,0,1], label=labels[k], color=colors[k])
                elif prop=='sh_pop_adi':
                    plt.plot(time_vec, x[:,1], label=labels[k], color=colors[k])
                else:
                    plt.plot(time_vec, x*units.au2ev*1000, label=labels[k], color=colors[k])
            if prop=='sh_pop_adi':
                ml_mctdh = np.loadtxt('reference/ML-MCTDH_n32.dat')
                plt.plot(ml_mctdh[:,0]/1000, ml_mctdh[:,1], label='Ref:ML-MCTDH', ls='dashed', color='black')
                plt.ylabel('S$_1$ Quantum Population')
            elif prop=='ave_decoherence_rates':
                plt.ylabel('Decoherence rate, 1/a.u.')
            else:
                plt.ylabel('Energy, meV')
            plt.xlim(0,xl)
            if c==1:
                plt.legend(fontsize=36, ncol=1, loc='center right')
            plt.xlabel('Time, ps')
            name = dec_method.replace('_REV23','')
#             if method=='FSSH2':
#                 plt.title(f'FSSH-2 with {name}', fontsize=35)
#             elif dec_method=='MFSD':
#                 plt.title(f'MFSD', fontsize=35)
#             else:
#                 plt.title(f'{method} with {name}', fontsize=35)
            plt.title(f'{method.upper()}-{name}, EDC, No SSY')
            c += 1
#         if prop=='sh_pop_adi':
#             title = 'Population dynamics, '
#         elif prop=='ave_decoherence_rates':
#             title = 'Average decoherence rates, '
#         elif prop=='Etot_ave':
#             title = 'Average total energy, '
#         elif prop=='Ekin_ave':
#             title = 'Average kinetic energy, '
#         elif prop=='Epot_ave':
#             title = 'Average potential energy, '
        #plt.suptitle(title+f'dt={dt}a.u.,ntraj={ntraj},elc_int={elec_int}, EDC - No SSY', fontsize=45)
        for xl in xls:
            print('Zooming in ', 0, xl)
#             for c3 in range(1,3):
#                 plt.subplot(1,2,c3)
            plt.xlim(0,xl)
            plt.tight_layout()
            plt.savefig(f'{method}_{dec_method}_SE_{prop}_EDC_no_SSY_xlim_{xl}_ntraj_{ntraj}_dt_{dt}_elec_int_{elec_int}.jpg', dpi=dpi) 

In [ ]:
print(eps_vals)

# Gu-Franco decoherence time

In [ ]:
# reorg_energies = [0.000000000125, 0.0000125, 0.000125, 0.0125, 1.25, 12.5, 125.0, 1250.0]
reorg_energies = [0.000000000125, 0.0000125, 0.0125, 1.25, 12.5]

reorg_energies_labels = []
for x in reorg_energies:
    mantissa_str, exponent_str = f"{x:.2e}".split('e')
    mantissa = float(mantissa_str)
    exponent = int(exponent_str)
    label = fr"${mantissa:.2f} \times 10^{{{exponent}}}$"
    reorg_energies_labels.append(label)

print(reorg_energies_labels)

In [ ]:
%matplotlib inline
colors = plt.cm.tab10.colors
colors = list(colors)
colors.append('darkblue')
plt.rcParams.update({'font.size': 42, 'axes.linewidth': 3, 
                     'xtick.major.width': 3, 'ytick.major.width': 3, 'lines.linewidth': 6.0})
# reorg_energies = [0.000125, 0.00125, 0.0125, 0.125, 1.25, 12.5]
# reorg_energies = [0.000000000125, 0.0000125, 0.000125, 0.0125, 1.25, 12.5, 125.0, 1250.0]
# reorg_energies_labels = [fr"${mantissa:.2f} \times 10^{{{exponent}}}$" 
#                          for mantissa, exponent in [f"{x:.2e}".split('e') for x in reorg_energies]]
# reorg_energies = [0.000125, 0.0125, 1.25, 125.0]
for prop in ['sh_pop_adi']:#,'ave_decoherence_rates','Etot_ave','Ekin_ave','Epot_ave']:
    print('Plotting ', prop)
    for method in ['FSSH','FSSH2','GFSH']:
        c = 1
        plt.figure(figsize=(3.21*4.5, 2.41*4))
        for dec_method in ['SDM']:
#             plt.subplot(1,2,c)
            print(dec_method, method)
            for c2, reorg_energy in enumerate(reorg_energies):
                file = f'all_methods_dec_wigner_saikat/{method.lower()}_{dec_method}_Gu_Franco_ntraj_{ntraj}_iter_0_dt_{dt}_temperature_300.0_reorg_energy_{reorg_energy}_istate_{istate}_rep_{rep}_elec_int_{elec_int}/mem_data.hdf'
                F = h5py.File(file)
                if dec_method=='DISH_REV23' and prop=='sh_pop_adi':
                    prop = 'se_pop_adi'
                else:
                    prop = 'sh_pop_adi'
                x = np.array(F[f'{prop}/data']) 
                time_vec = F['time/data'][:]*units.au2fs/1000
                F.close()
                if method=='FSSH2':
                    label = 'FSSH-2'
                else:
                    label = method
                if prop=='ave_decoherence_rates':
                    plt.plot(time_vec, x[:,0,1], label=f'$E_r$={reorg_energy}')
                    plt.ylabel('Decoherence rate, 1/a.u.')
                elif prop=='sh_pop_adi' or prop=='se_pop_adi':
                    plt.plot(time_vec, x[:,1], label=f'$E_r$={reorg_energies_labels[c2]}')
                    plt.ylabel('S$_1$ SH Population')
                else:
                    plt.plot(time_vec, x*units.au2ev*1000, label=label)
                    plt.ylabel('Energy, meV')
            plt.xlabel('Time, ps')
            name = dec_method.replace('_REV23','')
            plt.title(f'{method.upper()}-{name}, Gu-Franco, No SSY')
            c += 1
            if prop=='sh_pop_adi' or prop=='se_pop_adi':
                ml_mctdh = np.loadtxt('reference/ML-MCTDH_n32.dat')
                plt.plot(ml_mctdh[:,0]/1000, ml_mctdh[:,1], label='Ref:ML-MCTDH', ls='dashed', color='black') 
            plt.legend(fontsize=34, ncol=1, loc='center right')
        plt.tight_layout()
        for xl in xls:
            print('Zooming in ', 0, xl)
#             for c3 in range(1,3):
#                 plt.subplot(1,2,c3)
            plt.xlim(0,xl)
            plt.tight_layout()
            plt.savefig(f'{method}_{dec_method}_SH_{prop}_Gu_Franco_no_SSY_xlim_{xl}_ntraj_{ntraj}_dt_{dt}_elec_int_{elec_int}.jpg', dpi=dpi)

In [ ]:
%matplotlib inline
colors = plt.cm.tab10.colors
colors = list(colors)
colors.append('darkblue')
plt.rcParams.update({'font.size': 42, 'axes.linewidth': 3, 
                     'xtick.major.width': 3, 'ytick.major.width': 3, 'lines.linewidth': 6.0})
# reorg_energies = [0.000125, 0.00125, 0.0125, 0.125, 1.25, 12.5]
# reorg_energies = [0.000000000125, 0.0000125, 0.000125, 0.0125, 1.25, 12.5, 125.0, 1250.0]
# reorg_energies_labels = [fr"${mantissa:.2f} \times 10^{{{exponent}}}$" 
#                          for mantissa, exponent in [f"{x:.2e}".split('e') for x in reorg_energies]]
# reorg_energies = [0.000125, 0.0125, 1.25, 125.0]
for prop in ['sh_pop_adi']:#,'ave_decoherence_rates','Etot_ave','Ekin_ave','Epot_ave']:
    print('Plotting ', prop)
    for method in ['FSSH','FSSH2','GFSH']:
        c = 1
        plt.figure(figsize=(3.21*4.5, 2.41*4))
        for dec_method in ['SDM']:
#             plt.subplot(1,2,c)
            print(dec_method, method)
            for c2, reorg_energy in enumerate(reorg_energies):
                file = f'all_methods_dec_wigner_saikat/{method.lower()}_{dec_method}_Gu_Franco_ntraj_{ntraj}_iter_0_dt_{dt}_temperature_300.0_reorg_energy_{reorg_energy}_istate_{istate}_rep_{rep}_elec_int_{elec_int}/mem_data.hdf'
                F = h5py.File(file)
                if dec_method=='DISH_REV23' and prop=='sh_pop_adi':
                    prop = 'se_pop_adi'
                else:
                    prop = 'se_pop_adi'
                x = np.array(F[f'{prop}/data']) 
                time_vec = F['time/data'][:]*units.au2fs/1000
                F.close()
                if method=='FSSH2':
                    label = 'FSSH-2'
                else:
                    label = method
                if prop=='ave_decoherence_rates':
                    plt.plot(time_vec, x[:,0,1], label=f'$E_r$={reorg_energy}')
                    plt.ylabel('Decoherence rate, 1/a.u.')
                elif prop=='sh_pop_adi' or prop=='se_pop_adi':
                    plt.plot(time_vec, x[:,1], label=f'$E_r$={reorg_energies_labels[c2]}')
                    plt.ylabel('S$_1$ Quantum Population')
                else:
                    plt.plot(time_vec, x*units.au2ev*1000, label=label)
                    plt.ylabel('Energy, meV')
            plt.xlabel('Time, ps')
            name = dec_method.replace('_REV23','')
            plt.title(f'{method.upper()}-{name}, Gu-Franco, No SSY')
            c += 1
            if prop=='sh_pop_adi' or prop=='se_pop_adi':
                ml_mctdh = np.loadtxt('reference/ML-MCTDH_n32.dat')
                plt.plot(ml_mctdh[:,0]/1000, ml_mctdh[:,1], label='Ref:ML-MCTDH', ls='dashed', color='black') 
            plt.legend(fontsize=34, ncol=1, loc='center right')
        plt.tight_layout()
        for xl in xls:
            print('Zooming in ', 0, xl)
#             for c3 in range(1,3):
#                 plt.subplot(1,2,c3)
            plt.xlim(0,xl)
            plt.tight_layout()
            plt.savefig(f'{method}_{dec_method}_SE_{prop}_Gu_Franco_no_SSY_xlim_{xl}_ntraj_{ntraj}_dt_{dt}_elec_int_{elec_int}.jpg', dpi=dpi)

In [ ]:
%matplotlib inline
colors = plt.cm.tab10.colors
colors = list(colors)
colors.append('darkblue')
plt.rcParams.update({'font.size': 42, 'axes.linewidth': 3, 
                     'xtick.major.width': 3, 'ytick.major.width': 3, 'lines.linewidth': 6.0})
# reorg_energies = [0.000125, 0.00125, 0.0125, 0.125, 1.25, 12.5]
# reorg_energies = [0.000000000125, 0.0000125, 0.000125, 0.0125, 1.25, 12.5, 125.0, 1250.0]
# reorg_energies_labels = [fr"${mantissa:.2f} \times 10^{{{exponent}}}$" 
#                          for mantissa, exponent in [f"{x:.2e}".split('e') for x in reorg_energies]]
# reorg_energies = [0.000125, 0.0125, 1.25, 125.0]
for prop in ['sh_pop_adi']:#,'ave_decoherence_rates','Etot_ave','Ekin_ave','Epot_ave']:
    print('Plotting ', prop)
    for method in ['FSSH','FSSH2','GFSH']:
        c = 1
        plt.figure(figsize=(3.21*4.5, 2.41*4))
        for dec_method in ['DISH_REV23']:
#             plt.subplot(1,2,c)
            print(dec_method, method)
            for c2, reorg_energy in enumerate(reorg_energies):
                file = f'all_methods_dec_wigner_saikat/{method.lower()}_{dec_method}_Gu_Franco_ntraj_{ntraj}_iter_0_dt_{dt}_temperature_300.0_reorg_energy_{reorg_energy}_istate_{istate}_rep_{rep}_elec_int_{elec_int}/mem_data.hdf'
                F = h5py.File(file)
                if dec_method=='DISH_REV23' and prop=='sh_pop_adi':
                    prop = 'sh_pop_adi'
                else:
                    prop = 'sh_pop_adi'
                x = np.array(F[f'{prop}/data']) 
                time_vec = F['time/data'][:]*units.au2fs/1000
                F.close()
                if method=='FSSH2':
                    label = 'FSSH-2'
                else:
                    label = method
                if prop=='ave_decoherence_rates':
                    plt.plot(time_vec, x[:,0,1], label=f'$E_r$={reorg_energy}')
                    plt.ylabel('Decoherence rate, 1/a.u.')
                elif prop=='sh_pop_adi' or prop=='se_pop_adi':
                    plt.plot(time_vec, x[:,1], label=f'$E_r$={reorg_energies_labels[c2]}')
                    plt.ylabel('S$_1$ SH Population')
                else:
                    plt.plot(time_vec, x*units.au2ev*1000, label=label)
                    plt.ylabel('Energy, meV')
            plt.xlabel('Time, ps')
            name = dec_method.replace('_REV23','')
            plt.title(f'{method.upper()}-{name}, Gu-Franco, No SSY')
            c += 1
            if prop=='sh_pop_adi' or prop=='se_pop_adi':
                ml_mctdh = np.loadtxt('reference/ML-MCTDH_n32.dat')
                plt.plot(ml_mctdh[:,0]/1000, ml_mctdh[:,1], label='Ref:ML-MCTDH', ls='dashed', color='black') 
            plt.legend(fontsize=34, ncol=1, loc='center right')
        plt.tight_layout()
        for xl in xls:
            print('Zooming in ', 0, xl)
#             for c3 in range(1,3):
#                 plt.subplot(1,2,c3)
            plt.xlim(0,xl)
            plt.tight_layout()
            plt.savefig(f'{method}_{dec_method}_SH_{prop}_Gu_Franco_no_SSY_xlim_{xl}_ntraj_{ntraj}_dt_{dt}_elec_int_{elec_int}.jpg', dpi=dpi)

In [ ]:
%matplotlib inline
colors = plt.cm.tab10.colors
colors = list(colors)
colors.append('darkblue')
plt.rcParams.update({'font.size': 42, 'axes.linewidth': 3, 
                     'xtick.major.width': 3, 'ytick.major.width': 3, 'lines.linewidth': 6.0})
# reorg_energies = [0.000125, 0.00125, 0.0125, 0.125, 1.25, 12.5]
# reorg_energies = [0.000000000125, 0.0000125, 0.000125, 0.0125, 1.25, 12.5, 125.0, 1250.0]
# reorg_energies_labels = [fr"${mantissa:.2f} \times 10^{{{exponent}}}$" 
#                          for mantissa, exponent in [f"{x:.2e}".split('e') for x in reorg_energies]]
# reorg_energies = [0.000125, 0.0125, 1.25, 125.0]
for prop in ['sh_pop_adi']:#,'ave_decoherence_rates','Etot_ave','Ekin_ave','Epot_ave']:
    print('Plotting ', prop)
    for method in ['FSSH','FSSH2','GFSH']:
        c = 1
        plt.figure(figsize=(3.21*4.5, 2.41*4))
        for dec_method in ['DISH_REV23']:
#             plt.subplot(1,2,c)
            print(dec_method, method)
            for c2, reorg_energy in enumerate(reorg_energies):
                file = f'all_methods_dec_wigner_saikat/{method.lower()}_{dec_method}_Gu_Franco_ntraj_{ntraj}_iter_0_dt_{dt}_temperature_300.0_reorg_energy_{reorg_energy}_istate_{istate}_rep_{rep}_elec_int_{elec_int}/mem_data.hdf'
                F = h5py.File(file)
                if dec_method=='DISH_REV23' and prop=='sh_pop_adi':
                    prop = 'se_pop_adi'
                else:
                    prop = 'sh_pop_adi'
                x = np.array(F[f'{prop}/data']) 
                time_vec = F['time/data'][:]*units.au2fs/1000
                F.close()
                if method=='FSSH2':
                    label = 'FSSH-2'
                else:
                    label = method
                if prop=='ave_decoherence_rates':
                    plt.plot(time_vec, x[:,0,1], label=f'$E_r$={reorg_energy}')
                    plt.ylabel('Decoherence rate, 1/a.u.')
                elif prop=='sh_pop_adi' or prop=='se_pop_adi':
                    plt.plot(time_vec, x[:,1], label=f'$E_r$={reorg_energies_labels[c2]}')
                    plt.ylabel('S$_1$ Quantum Population')
                else:
                    plt.plot(time_vec, x*units.au2ev*1000, label=label)
                    plt.ylabel('Energy, meV')
            plt.xlabel('Time, ps')
            name = dec_method.replace('_REV23','')
            plt.title(f'{method.upper()}-{name}, Gu-Franco, No SSY')
            c += 1
            if prop=='sh_pop_adi' or prop=='se_pop_adi':
                ml_mctdh = np.loadtxt('reference/ML-MCTDH_n32.dat')
                plt.plot(ml_mctdh[:,0]/1000, ml_mctdh[:,1], label='Ref:ML-MCTDH', ls='dashed', color='black') 
            plt.legend(fontsize=34, ncol=1, loc='center right')
        plt.tight_layout()
        for xl in xls:
            print('Zooming in ', 0, xl)
#             for c3 in range(1,3):
#                 plt.subplot(1,2,c3)
            plt.xlim(0,xl)
            plt.tight_layout()
            plt.savefig(f'{method}_{dec_method}_SE_{prop}_Gu_Franco_no_SSY_xlim_{xl}_ntraj_{ntraj}_dt_{dt}_elec_int_{elec_int}.jpg', dpi=dpi)

# SHXF results with different wavepacket width

In [ ]:
%matplotlib inline
plt.rcParams.update({'font.size': 42, 'axes.linewidth': 3, 'lines.linewidth': 6.0})
# for prop in ['sh_pop_adi']:#,'ave_decoherence_rates','Etot_ave','Ekin_ave','Epot_ave']:
for prop in ['sh_pop_adi']:
    print('Plotting ', prop)
    plt.figure(figsize=(3.21*4.5*2, 2.41*4))
    #plt.figure(figsize=(3.21*9,2.41*3.5))
    #colors1 = []
    colors = plt.cm.tab20.colors
    F = h5py.File('FSSH/time.hdf')
    time_vec = F['time/data'][:]*units.au2fs/1000
    F.close()
    for c, method in enumerate(['FSSH','FSSH2','GFSH']):
        print(method)
        plt.subplot(1,3,c+1) 
        #for wpwidth_scale in [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 1.0, 2.0, 4.0]:
#        for wpwidth_scale in [0.05, 0.1, 0.25, 0.5, 1.0, 2.0, 4.0]:
#         for wpwidth_scale in [0.05, 0.1, 0.25, 0.5, 1.0, 2.0, 4.0]:
        for wpwidth_scale in [0.05, 0.1, 1.0, 2.0, 4.0]:
            file = f'all_methods_dec_wigner_saikat/{method.lower()}_SHXF_ntraj_{ntraj}_iter_0_dt_{dt}_wpwidth_scale_{wpwidth_scale}_istate_{istate}_rep_{rep}_elec_int_{elec_int}/mem_data.hdf'
            #file = 'all_methods_dec/gfsh_SHXF_ntraj_1000_iter_0_dt_5.0_wpwidth_scale_0.1_istate_1_rep_1_elec_int_2/mem_data.hdf'
            F = h5py.File(file)
            if prop=='sh_pop_adi':
                x = np.array(F[f'sh_pop_adi/data'])
            else:
                x = np.array(F[f'{prop}/data'])
            time_vec = F['time/data'][:]*units.au2fs/1000
            F.close()
            if prop=='ave_decoherence_rates':
                plt.plot(time_vec, x[:,0,1], label=f'{wpwidth_scale}$\\sigma_q$')#, color=colors[k])
                plt.ylabel('Decoherence rate, 1/a.u.')
            elif prop=='sh_pop_adi':
                plt.plot(time_vec, x[:,1], label=f'{wpwidth_scale}$\\sigma_q$')#, color=colors[k])
                plt.ylabel('S$_1$ SH Population')
            else:
                plt.plot(time_vec, x*units.au2ev*1000, label=f'{wpwidth_scale}$\\sigma_q$')#, color=colors[k])
                plt.ylabel('Energy, meV')
            #plt.plot(time_vec, sh_pop[:,1], label=f'{wpwidth_scale}$\\sigma_q$')
        if prop=='sh_pop_adi':
            ml_mctdh = np.loadtxt('reference/ML-MCTDH_n32.dat')
            plt.plot(ml_mctdh[:,0]/1000, ml_mctdh[:,1], label='ML-MCTDH', ls='dashed', color='black')
        #plt.xlim(0,50)
        if c==2:
            plt.legend(fontsize=34, ncol=1, loc='upper right')
        plt.xlabel('Time, ps')
        if method=='FSSH2':
            plt.title('FSSH-2')
        else:    
            plt.title(method)
    if prop=='sh_pop_adi':
        title = 'Population dynamics, '
    elif prop=='ave_decoherence_rates':
        title = 'Average decoherence rates, '
    elif prop=='Etot_ave':
        title = 'Average total energy, '
    elif prop=='Ekin_ave':
        title = 'Average kinetic energy, '
    elif prop=='Epot_ave':
        title = 'Average potential energy, '
#    plt.suptitle(title+f'dt={dt}a.u.,ntraj={ntraj},elc_int={elec_int}, SHXF - No SSY', fontsize=45)
    plt.suptitle("$S_1$ "+title+f'SHXF, No SSY')
    plt.tight_layout()
    for xl in xls:
        print('Zooming in ', 0, xl)
        for c3 in range(1,4):
            plt.subplot(1,3,c3)
            plt.xlim(0,xl)
        plt.tight_layout()
        plt.savefig(f'{prop}_SHXF_no_SSY_xlim_{xl}_ntraj_{ntraj}_dt_{dt}_elec_int_{elec_int}.jpg', dpi=dpi)

In [ ]:
F = h5py.File(f'all_methods_dec_wigner_saikat/fssh__ntraj_{ntraj}_iter_0_dt_{dt}_istate_{istate}_rep_{rep}_elec_int_{elec_int}/mem_data.hdf')
time_vec = np.array(F['time/data'][:])*units.au2fs/1000
ml_mctdh = np.loadtxt('reference/ML-MCTDH_n32.dat')
# simulation_time_vec = np.arange(x.shape[0]) * units.au2fs * dt
simulation_time_vec = time_vec * 1000
reference_time_vec = ml_mctdh[:, 0]

# Use searchsorted for efficient nearest-neighbor indexing
indices = np.searchsorted(simulation_time_vec, reference_time_vec)

# Adjust indices to find the *closest* not just the next one
indices = np.clip(indices, 1, len(reference_time_vec) - 1)
left = simulation_time_vec[indices - 1]
right = simulation_time_vec[indices]
comparing_indices = indices - (np.abs(reference_time_vec - left) < np.abs(reference_time_vec - right))

print(reference_time_vec)
print(len(comparing_indices))

In [ ]:
print(np.average(np.abs(ml_mctdh[0:len(comparing_indices),0]-simulation_time_vec[comparing_indices])))

# Now the overall error measurements (the main Figure)

In [ ]:
all_files = glob.glob(f'all_methods_dec_wigner_saikat/*ntraj_{ntraj}*dt_{dt}*istate_{istate}_rep_{rep}_elec_int_{elec_int}/*hdf')
all_files_no_ssy = [file for file in all_files if 'SSY' not in file]
ml_mctdh = np.loadtxt('reference/ML-MCTDH_n32.dat')
print(len(all_files_no_ssy))
all_errors_no_ssy = []
for file in all_files_no_ssy:
    try:
        F = h5py.File(file)
        if ('MFSD' in file) or ('DISH' in file):
            #pop = np.array(F['se_pop_adi/data'])
            pop = np.array(F['sh_pop_adi/data'])
        else:
            pop = np.array(F['sh_pop_adi/data'])
        F.close()
        #error = np.average(np.abs(ml_mctdh[0:len(comparing_indices),1]-pop[comparing_indices,1]))
        tmp1 = ml_mctdh[0:len(comparing_indices),1]-pop[comparing_indices,1]
        error = np.sqrt(np.average(np.multiply(tmp1, tmp1)))
        all_errors_no_ssy.append(error)
    except:
        all_errors_no_ssy.append(100.0)

        #pass
print(len(all_errors_no_ssy))

In [ ]:
# print(all_errors_no_ssy)
all_errors_no_ssy = np.array(all_errors_no_ssy)
sort_args = np.argsort(all_errors_no_ssy)
sorted_all_errors_no_ssy = all_errors_no_ssy[sort_args]
sorted_all_files_no_ssy = np.array(all_files_no_ssy)[sort_args]
print(sorted_all_files_no_ssy)
print(sorted_all_errors_no_ssy[[0,1,2,3,4,5,-1]])

In [ ]:
# for i in range(len(sorted_all_files_no_ssy)):
#     if 'DISH' in sorted_all_files_no_ssy[i] and 'SCHW1' in sorted_all_files_no_ssy[i]:
#         print(sorted_all_files_no_ssy[i], sorted_all_errors_no_ssy[i])
# # for i in range(len(all_files_no_ssy)):
# #     if 'DISH' in all_files_no_ssy[i] and 'SCHW1' in all_files_no_ssy[i]:
# #         print(all_files_no_ssy[i], all_errors_no_ssy[i])

In [ ]:
# !pip install colorcet

In [ ]:
import colorcet as cc

In [ ]:
colors = cc.glasbey

In [ ]:
print(cc.gla[0])

In [ ]:
%matplotlib inline
plt.rcParams.update({'font.size': 35, 'axes.linewidth': 3, 'lines.linewidth': 3.0})
plt.figure(figsize=(3.21*8,2.41*6))
#colors1 = []
colors = list(plt.cm.tab20.colors) + list(cc.glasbey[22:28]) + list(cc.glasbey[29:38])
# colors = cc.glasbey[[1,2,3,]]
selected_cases_no_ssy = []
selected_errors_no_ssy = []
main_fig_labels_no_ssy = []
files_indices_no_ssy = []
for c1, method in enumerate(['FSSH','FSSH2','GFSH']):
    #print(method)
    #for dec_method in ['','BCSH','SHXF','ID-A','MFSD','SDM','DISH']:
    for dec_method in ['','BCSH','SHXF','ID-A','SDM','DISH']:
        if dec_method=='SDM' or dec_method=='DISH':
            if dec_method=='SDM':
                dec_time_methods_ = ['SCHW2','EDC','Gu_Franco']
            else:
                dec_time_methods_ = ['SCHW1','SCHW2','EDC','Gu_Franco']
            for dec_time_method in dec_time_methods_:
                if dec_method=='DISH':
                    combined_method = method.lower()+'_'+dec_method+'_REV23_'+dec_time_method
                else:
                    combined_method = method.lower()+'_'+dec_method+'_'+dec_time_method
                for i in range(len(sorted_all_files_no_ssy)):
                    if combined_method in sorted_all_files_no_ssy[i]:
                        selected_cases_no_ssy.append( sorted_all_files_no_ssy[i])
                        selected_errors_no_ssy.append(sorted_all_errors_no_ssy[i])
                        files_indices_no_ssy.append(i)
                        if dec_method=='':
                            main_fig_labels_no_ssy.append(method)
                        elif dec_method=='MFSD' and method=='FSSH2':
                            main_fig_labels_no_ssy.append(dec_method)
                        else:
                            if dec_time_method=='SCHW1':
                                main_fig_labels_no_ssy.append(method+'-'+dec_method+'-Schw1')
                            if dec_time_method=='SCHW2':
                                main_fig_labels_no_ssy.append(method+'-'+dec_method+'-Schw2')
                            if dec_time_method=='EDC':
                                main_fig_labels_no_ssy.append(method+'-'+dec_method+'-EDC')
                            if dec_time_method=='Gu_Franco':
                                main_fig_labels_no_ssy.append(method+'-'+dec_method+'-GF')
                    #print(main_fig_labels)
                        break
        else:
            combined_method = method.lower()+'_'+dec_method+'_'
            for i in range(len(sorted_all_files_no_ssy)):
                if combined_method in sorted_all_files_no_ssy[i]:
                    selected_cases_no_ssy.append(sorted_all_files_no_ssy[i])
                    selected_errors_no_ssy.append(sorted_all_errors_no_ssy[i])
                    files_indices_no_ssy.append(i)
                    if dec_method=='':
                        main_fig_labels_no_ssy.append(method)
                    elif dec_method=='MFSD' and method=='FSSH2':
                        main_fig_labels_no_ssy.append(dec_method)
                    elif dec_method=='ID-A':
                        main_fig_labels_no_ssy.append(method+'-IDA')
                    else:
                        main_fig_labels_no_ssy.append(method+'-'+dec_method)
                    #print(main_fig_labels)

                    break
# print('selected_cases',selected_cases)
# print('selected_errors',selected_errors)
print(len(selected_cases_no_ssy))
print(main_fig_labels_no_ssy, len(main_fig_labels_no_ssy))
sorted_indices_no_ssy = np.argsort(selected_errors_no_ssy)
plt.bar(np.array(main_fig_labels_no_ssy)[sorted_indices_no_ssy], 
        np.array(selected_errors_no_ssy)[sorted_indices_no_ssy], color=colors)
# indices_1 = np.argsort(main_fig_errors)
# plt.bar(main_fig_labels[indices_1], main_fig_errors[indices_1], color=colors)
# plt.ylabel('$\\epsilon_{pop}$')
plt.ylabel('Root mean square deviation')
plt.tick_params(axis='y', width=3)
plt.tick_params(axis='y', which='minor', width=2.5, length=7)
plt.tick_params(axis='y', which='major', width=3.5, length=13)
for i, value in enumerate(np.array(selected_errors_no_ssy)[sorted_indices_no_ssy]):
    if 0.2<value<0.22:
        plt.text(i, value-0.06, f'{value:.4f}', ha='center', va='bottom', fontsize=30, rotation=90)
    elif 0.22<value<0.25:
        plt.text(i, value-0.08, f'{value:.4f}', ha='center', va='bottom', fontsize=30, rotation=90)
    elif 0.39<value<0.5:
        plt.text(i, value-0.27, f'{value:.4f}', ha='center', va='bottom', fontsize=30, rotation=90)
    elif 0.5<value<0.6:
        plt.text(i, value-0.35, f'{value:.4f}', ha='center', va='bottom', fontsize=30, rotation=90)
    elif 0.6<value:
        plt.text(i, value-0.4, f'{value:.4f}', ha='center', va='bottom', fontsize=30, rotation=90)
    else:
        plt.text(i, value + 0.002, f'{value:.4f}', ha='center', va='bottom', fontsize=30, rotation=90)
plt.yscale('log')
plt.xticks(rotation=90)  # or 90 for vertical
plt.title('Population error, Wigner, No SSY')
plt.tight_layout()
plt.savefig(f'new_main_fig_error_no_ssy_elec_int_{elec_int}_wigner.jpg',dpi=dpi)

In [ ]:
np.save('wigner_0K_error_labels.npy', np.array(main_fig_labels_no_ssy)[sorted_indices_no_ssy])
np.save('wigner_0K_errors.npy', np.array(selected_errors_no_ssy)[sorted_indices_no_ssy])

In [ ]:
all_files = glob.glob(f'all_methods_dec_wigner_saikat/*ntraj_{ntraj}*dt_{dt}*istate_{istate}_rep_{rep}_elec_int_{elec_int}/*hdf')
all_files_ssy = [file for file in all_files if 'SSY' in file]
ml_mctdh = np.loadtxt('reference/ML-MCTDH_n32.dat')
print(len(all_files_ssy))
all_errors_ssy = []
for file in all_files_ssy:
    try:
        F = h5py.File(file)
        if ('MFSD' in file) or ('DISH' in file):
            pop = np.array(F['sh_pop_adi/data'])
        else:
            pop = np.array(F['sh_pop_adi/data'])
        F.close()
        tmp1 = ml_mctdh[0:len(comparing_indices),1]-pop[comparing_indices,1]
        error = np.sqrt(np.average(np.multiply(tmp1, tmp1)))
        all_errors_ssy.append(error)
    except:
        all_errors_ssy.append(100.0)

        #pass
print(len(all_errors_ssy))

In [ ]:
# print(all_errors_no_ssy)
all_errors_ssy = np.array(all_errors_ssy)
sort_args = np.argsort(all_errors_ssy)
sorted_all_errors_ssy = all_errors_ssy[sort_args]
sorted_all_files_ssy = np.array(all_files_ssy)[sort_args]
print(sorted_all_files_ssy)
print(sorted_all_errors_ssy[[0,1,2,3,4,5,-1]])

In [ ]:
import pandas as pd

In [ ]:
a11 = np.array(main_fig_labels_no_ssy)[sorted_indices_no_ssy]
a12 = np.array(sorted_all_files_no_ssy)[files_indices_no_ssy]
print(a11)
print(a12[sorted_indices_no_ssy])

In [ ]:
a21 = np.array(main_fig_labels_with_ssy)[sorted_indices_with_ssy]
a22 = np.array(sorted_all_files_ssy)[files_indices_with_ssy]
print(a21)
print(a22[sorted_indices_with_ssy])

In [ ]:
a21 = np.array(main_fig_labels_with_ssy)[sorted_indices_no_ssy]
a22 = np.array(sorted_all_files_ssy)[files_indices_with_ssy]
print(a21)
print(a22[sorted_indices_no_ssy])

In [ ]:
%matplotlib inline
plt.rcParams.update({'font.size': 35, 'axes.linewidth': 3, 'lines.linewidth': 3.0})
plt.figure(figsize=(3.21*8,2.41*6))
#colors1 = []
# colors = plt.cm.tab20.colors+plt.cm.tab10.colors
selected_cases_with_ssy = []
selected_errors_with_ssy = []
main_fig_labels_with_ssy = []
files_indices_with_ssy = []
for c1, method in enumerate(['FSSH','FSSH2','GFSH']):
    #print(method)
    #for dec_method in ['','BCSH','SHXF','ID-A','MFSD','SDM','DISH']:
    for dec_method in ['','BCSH','SHXF','ID-A','SDM','DISH']:
        if dec_method=='SDM' or dec_method=='DISH':
            if dec_method=='SDM':
                dec_time_methods_ = ['SCHW2','EDC','Gu_Franco']
            else:
                dec_time_methods_ = ['SCHW1','SCHW2','EDC','Gu_Franco']
            for dec_time_method in dec_time_methods_:
                if dec_method=='DISH':
                    combined_method = method.lower()+'_'+dec_method+'_REV23_'+dec_time_method
                else:
                    combined_method = method.lower()+'_'+dec_method+'_'+dec_time_method
                for i in range(len(sorted_all_files_ssy)):
                    if combined_method in sorted_all_files_ssy[i]:
                        selected_cases_with_ssy.append(sorted_all_files_ssy[i])
                        selected_errors_with_ssy.append(sorted_all_errors_ssy[i])
                        files_indices_with_ssy.append(i)
                        if dec_method=='':
                            main_fig_labels_with_ssy.append(method)
                        elif dec_method=='MFSD' and method=='FSSH2':
                            main_fig_labels_with_ssy.append(dec_method)
                        else:
                            if dec_time_method=='SCHW1':
                                main_fig_labels_with_ssy.append(method+'-'+dec_method+'-Schw1')
                            if dec_time_method=='SCHW2':
                                main_fig_labels_with_ssy.append(method+'-'+dec_method+'-Schw2')
                            if dec_time_method=='EDC':
                                main_fig_labels_with_ssy.append(method+'-'+dec_method+'-EDC')
                            if dec_time_method=='Gu_Franco':
                                main_fig_labels_with_ssy.append(method+'-'+dec_method+'-GF')
                    #print(main_fig_labels)
                        break
        else:
            combined_method = method.lower()+'_'+dec_method+'_'
            for i in range(len(sorted_all_files_ssy)):
                if combined_method in sorted_all_files_ssy[i]:
                    selected_cases_with_ssy.append(sorted_all_files_ssy[i])
                    selected_errors_with_ssy.append(sorted_all_errors_ssy[i])
                    files_indices_with_ssy.append(i)
                    if dec_method=='':
                        main_fig_labels_with_ssy.append(method)
                    elif dec_method=='MFSD' and method=='FSSH2':
                        main_fig_labels_with_ssy.append(dec_method)
                    elif dec_method=='ID-A':
                        main_fig_labels_with_ssy.append(method+'-IDA')
                    else:
                        main_fig_labels_with_ssy.append(method+'-'+dec_method)
                    #print(main_fig_labels)

                    break
# print('selected_cases',selected_cases)
# print('selected_errors',selected_errors)
# print(len(selected_cases))
print(main_fig_labels_with_ssy, len(main_fig_labels_with_ssy))
sorted_indices_with_ssy = np.argsort(selected_errors_with_ssy)
# plt.bar(np.array(main_fig_labels)[sorted_indices], np.array(selected_errors)[sorted_indices], color=colors)
plt.bar(np.array(main_fig_labels_with_ssy)[sorted_indices_no_ssy], 
        np.array(selected_errors_with_ssy)[sorted_indices_no_ssy], color=colors)
# indices_1 = np.argsort(main_fig_errors)
# plt.bar(main_fig_labels[indices_1], main_fig_errors[indices_1], color=colors)
plt.ylabel('$\\epsilon_{pop}$')
plt.tick_params(axis='y', width=3)
plt.tick_params(axis='y', which='minor', width=2.5, length=7)
plt.tick_params(axis='y', which='major', width=3.5, length=13)
# for i, value in enumerate(np.array(selected_errors)[sorted_indices]):
for i, value in enumerate(np.array(selected_errors_with_ssy)[sorted_indices_no_ssy]):
    if 0.1<value<0.15:
        plt.text(i, value-0.07, f'{value:.4f}', ha='center', va='bottom', fontsize=30, rotation=90)
    elif 0.15<value<0.19:
        plt.text(i, value-0.1, f'{value:.4f}', ha='center', va='bottom', fontsize=30, rotation=90)
    elif 0.19<value<0.3:
        plt.text(i, value-0.16, f'{value:.4f}', ha='center', va='bottom', fontsize=30, rotation=90)
    elif 0.3<value<0.39:
        plt.text(i, value-0.19, f'{value:.4f}', ha='center', va='bottom', fontsize=30, rotation=90)
    elif 0.39<value<0.5:
        plt.text(i, value-0.29, f'{value:.4f}', ha='center', va='bottom', fontsize=30, rotation=90)
    elif 0.5<value<0.6:
        plt.text(i, value-0.36, f'{value:.4f}', ha='center', va='bottom', fontsize=30, rotation=90)
    elif 0.6<value:
        plt.text(i, value-0.42, f'{value:.4f}', ha='center', va='bottom', fontsize=30, rotation=90)
    else:
        plt.text(i, value + 0.002, f'{value:.4f}', ha='center', va='bottom', fontsize=30, rotation=90)
plt.yscale('log')
plt.xticks(rotation=90)  # or 90 for vertical
plt.title('Population error, With SSY')
plt.tight_layout()
plt.savefig(f'new_main_fig_error_with_ssy_elec_int_{elec_int}.jpg',dpi=dpi)

In [ ]:
%matplotlib inline
plt.rcParams.update({'font.size': 35, 'axes.linewidth': 3, 'lines.linewidth': 3.0})
plt.figure(figsize=(3.21*8,2.41*6))
#colors1 = []
# colors = plt.cm.tab20.colors+plt.cm.tab10.colors
selected_cases = []
selected_errors = []
main_fig_labels = []
for c1, method in enumerate(['FSSH','FSSH2','GFSH']):
    #print(method)
    #for dec_method in ['','BCSH','SHXF','ID-A','MFSD','SDM','DISH']:
    for dec_method in ['','BCSH','SHXF','ID-A','SDM','DISH']:
        if dec_method=='SDM' or dec_method=='DISH':
            if dec_method=='SDM':
                dec_time_methods_ = ['SCHW2','EDC','Gu_Franco']
            else:
                dec_time_methods_ = ['SCHW1','SCHW2','EDC','Gu_Franco']
            for dec_time_method in dec_time_methods_:
                if dec_method=='DISH':
                    combined_method = method.lower()+'_'+dec_method+'_REV23_'+dec_time_method
                else:
                    combined_method = method.lower()+'_'+dec_method+'_'+dec_time_method
                for i in range(len(sorted_all_files_ssy)):
                    if combined_method in sorted_all_files_ssy[i]:
                        selected_cases.append(sorted_all_files_ssy[i])
                        selected_errors.append(sorted_all_errors_ssy[i])
                        if dec_method=='':
                            main_fig_labels.append(method)
                        elif dec_method=='MFSD' and method=='FSSH2':
                            main_fig_labels.append(dec_method)
                        else:
                            if dec_time_method=='SCHW1':
                                main_fig_labels.append(method+'-'+dec_method+'-Schw1')
                            if dec_time_method=='SCHW2':
                                main_fig_labels.append(method+'-'+dec_method+'-Schw2')
                            if dec_time_method=='EDC':
                                main_fig_labels.append(method+'-'+dec_method+'-EDC')
                            if dec_time_method=='Gu_Franco':
                                main_fig_labels.append(method+'-'+dec_method+'-GF')
                    #print(main_fig_labels)
                        break
        else:
            combined_method = method.lower()+'_'+dec_method+'_'
            for i in range(len(sorted_all_files_ssy)):
                if combined_method in sorted_all_files_ssy[i]:
                    selected_cases.append(sorted_all_files_ssy[i])
                    selected_errors.append(sorted_all_errors_ssy[i])
                    if dec_method=='':
                        main_fig_labels.append(method)
                    elif dec_method=='MFSD' and method=='FSSH2':
                        main_fig_labels.append(dec_method)
                    elif dec_method=='ID-A':
                        main_fig_labels.append(method+'-IDA')
                    else:
                        main_fig_labels.append(method+'-'+dec_method)
                    #print(main_fig_labels)

                    break
# print('selected_cases',selected_cases)
# print('selected_errors',selected_errors)
# print(len(selected_cases))
print(main_fig_labels, len(main_fig_labels))
# sorted_indices = np.argsort(selected_errors)
# plt.bar(np.array(main_fig_labels)[sorted_indices], np.array(selected_errors)[sorted_indices], color=colors)
plt.bar(np.array(main_fig_labels)[sorted_indices_no_ssy], np.array(selected_errors)[sorted_indices_no_ssy], color=colors)
# indices_1 = np.argsort(main_fig_errors)
# plt.bar(main_fig_labels[indices_1], main_fig_errors[indices_1], color=colors)
plt.ylabel('$\\epsilon_{pop}$')
plt.tick_params(axis='y', width=3)
plt.tick_params(axis='y', which='minor', width=2.5, length=7)
plt.tick_params(axis='y', which='major', width=3.5, length=13)
# for i, value in enumerate(np.array(selected_errors)[sorted_indices]):
for i, value in enumerate(np.array(selected_errors)[sorted_indices_no_ssy]):
    if 0.1<value<0.15:
        plt.text(i, value-0.07, f'{value:.4f}', ha='center', va='bottom', fontsize=30, rotation=90)
    elif 0.15<value<0.19:
        plt.text(i, value-0.1, f'{value:.4f}', ha='center', va='bottom', fontsize=30, rotation=90)
    elif 0.19<value<0.3:
        plt.text(i, value-0.16, f'{value:.4f}', ha='center', va='bottom', fontsize=30, rotation=90)
    elif 0.3<value<0.39:
        plt.text(i, value-0.19, f'{value:.4f}', ha='center', va='bottom', fontsize=30, rotation=90)
    elif 0.39<value<0.5:
        plt.text(i, value-0.29, f'{value:.4f}', ha='center', va='bottom', fontsize=30, rotation=90)
    elif 0.5<value<0.6:
        plt.text(i, value-0.36, f'{value:.4f}', ha='center', va='bottom', fontsize=30, rotation=90)
    elif 0.6<value:
        plt.text(i, value-0.42, f'{value:.4f}', ha='center', va='bottom', fontsize=30, rotation=90)
    else:
        plt.text(i, value + 0.002, f'{value:.4f}', ha='center', va='bottom', fontsize=30, rotation=90)
plt.yscale('log')
plt.xticks(rotation=90)  # or 90 for vertical
plt.title('Population error, With SSY')
plt.tight_layout()
plt.savefig(f'new_main_fig_error_with_ssy_elec_int_{elec_int}.jpg',dpi=dpi)

# July
# Error figuers for SDM and DISH methods for different decoherence methods

# For Schwartz 1, A values

In [ ]:
%matplotlib inline
plt.rcParams.update({'font.size': 45, 'axes.linewidth': 3, 'lines.linewidth': 7.0})
plt.figure(figsize=(3.21*6,2.41*4))
A_vals = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0, 1000000.0, 10000000.0, 100000000.0, 1000000000.0]
for dec_method in ['DISH']:
    for method in ['fssh','fssh2','gfsh']:
        plot_errors = []
        for A in A_vals:
            for i in range(len(all_files_no_ssy)):
                #if all(x in all_files_no_ssy[i] for x in [f'{method}_', f'{dec_method}_REV23', 'SCHW1', f'{A}']):
                if dec_method in all_files_no_ssy[i] and f'{method}_' in all_files_no_ssy[i] and \
                'SCHW1' in all_files_no_ssy[i] and f'{A}' in all_files_no_ssy[i]:
                    #break
                    print(all_files_no_ssy[i], all_errors_no_ssy[i])
                    plot_errors.append(all_errors_no_ssy[i])
                    break
            #print(A, all_files_no_ssy[i],  all_errors_no_ssy[i])
        plt.plot(A_vals, plot_errors, marker='s', markersize=20, label=method.upper()+'-'+dec_method)
        # plt.scatter(A_vals, plot_errors, color='red', marker='s', s=1000)
# plt.legend(fontsize=30, ncol=2)
plt.legend(fontsize=34, ncol=1)
plt.xscale('log')
# plt.yscale('log')
# plt.title(f'dt={dt}a.u.,ntraj={ntraj},elc_int={elec_int}, Schwartz 1 - No SSY', fontsize=35)
plt.title(f'Schwartz 1 - No SSY')#, fontsize=35)
plt.xlabel('$A_k$')
plt.ylabel('$\\epsilon_{pop}$')
plt.tight_layout()
plt.savefig(f'schwartz1_Ak_elec_int_{elec_int}_no_ssy.jpg', dpi=600)

# For Schwartz 2, A values

In [ ]:
%matplotlib inline
plt.rcParams.update({'font.size': 45, 'axes.linewidth': 3, 'lines.linewidth': 7.0})
plt.figure(figsize=(3.21*6,2.41*4))
A_vals = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0, 1000000.0, 10000000.0, 100000000.0, 1000000000.0]
for dec_method in ['SDM','DISH']:
    for method in ['fssh','fssh2','gfsh']:
        plot_errors = []
        for A in A_vals:
            for i in range(len(all_files_no_ssy)):
                if all(x in all_files_no_ssy[i] for x in [f'{method}_', f'{dec_method}', 'SCHW2', f'{A}']):
                    plot_errors.append(all_errors_no_ssy[i])
                    #print([f'{method}_', f'{dec_method}', 'SCHW2', f'{A}'])
                    #break
            #print(A, all_files_no_ssy[i],  all_errors_no_ssy[i])
        plt.plot(A_vals, plot_errors, marker='s', markersize=20, label=method.upper()+'-'+dec_method)
        # plt.scatter(A_vals, plot_errors, color='red', marker='s', s=1000)
# plt.legend(fontsize=30)
plt.legend(fontsize=34, ncol=2)
plt.xscale('log')
# plt.yscale('log')
# plt.title(f'dt={dt}a.u.,ntraj={ntraj},elc_int={elec_int}, Schwartz 2 - No SSY', fontsize=35)
plt.title(f'Schwartz 2 - No SSY')#, fontsize=35)
plt.xlabel('$A_k$')
plt.ylabel('$\\epsilon_{pop}$')
plt.tight_layout()
plt.savefig(f'schwartz2_Ak_elec_int_{elec_int}_no_ssy.jpg', dpi=600)

# For EDC, $\epsilon$ values

In [ ]:
%matplotlib inline
plt.rcParams.update({'font.size': 45, 'axes.linewidth': 3, 'lines.linewidth': 7.0})
plt.figure(figsize=(3.21*6,2.41*4))
eps_vals = [0.01, 0.05, 0.1, 0.4, 1.0,10.0, 50.0, 100.0, 500.0, 1000.0]
for dec_method in ['SDM','DISH']:
    for method in ['fssh','fssh2','gfsh']:
        plot_errors = []
        for eps in eps_vals:
            for i in range(len(all_files_no_ssy)):
                if all(x in all_files_no_ssy[i] for x in [f'{method}_', f'{dec_method}', 'EDC', f'{eps}']):
                    break
            plot_errors.append(all_errors_no_ssy[i])
            #print(A, all_files_no_ssy[i],  all_errors_no_ssy[i])
        plt.plot(eps_vals, plot_errors, marker='s', markersize=20, label=method.upper()+'-'+dec_method)
        # plt.scatter(A_vals, plot_errors, color='red', marker='s', s=1000)
plt.legend(fontsize=34, ncol=2)
plt.xscale('log')
# plt.yscale('log')
plt.title(f'dt={dt}a.u.,ntraj={ntraj},elc_int={elec_int}, EDC - No SSY', fontsize=35)
plt.title(f'EDC - No SSY')#, fontsize=35)
plt.xlabel('$\\epsilon$, Ha')
plt.ylabel('$\\epsilon_{pop}$')
plt.tight_layout()
plt.savefig(f'edc_epsilon_elec_int_{elec_int}_no_ssy.jpg', dpi=600)

In [ ]:
%matplotlib inline
plt.rcParams.update({'font.size': 45, 'axes.linewidth': 3, 'lines.linewidth': 7.0})
plt.figure(figsize=(3.21*6,2.41*4))
# E_rs = [0.000125,0.00125,0.0125,0.125,1.25,12.5]
E_rs = [0.000000000125, 0.0000125, 0.0125, 1.25, 12.5]
for dec_method in ['SDM','DISH']:
    for method in ['fssh','fssh2','gfsh']:
        plot_errors = []
        for E_r in E_rs:
            for i in range(len(all_files_no_ssy)):
                if all(x in all_files_no_ssy[i] for x in [f'{method}_', f'{dec_method}', 'Gu', f'{E_r}']):
                    break
            plot_errors.append(all_errors_no_ssy[i])
            #print(A, all_files_no_ssy[i],  all_errors_no_ssy[i])
        plt.plot(E_rs, plot_errors, marker='s', markersize=20, label=method.upper()+'-'+dec_method)
        # plt.scatter(A_vals, plot_errors, color='red', marker='s', s=1000)
plt.legend(fontsize=34, ncol=2)
plt.xscale('log')
# plt.yscale('log')
# plt.title(f'dt={dt}a.u.,ntraj={ntraj},elc_int={elec_int}, EDC - No SSY', fontsize=35)
plt.title(f'Gu-Franco - No SSY')#, fontsize=35)
plt.xlabel('$E_r$, Ha')
plt.ylabel('$\\epsilon_{pop}$')
plt.tight_layout()
plt.savefig(f'gu-franco_elec_int_{elec_int}_no_ssy.jpg', dpi=600)